In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import seaborn as sns
from matplotlib import rc
rc('font', family='AppleGothic')
import matplotlib.pyplot as plt
%matplotlib inline

## 데이터 로드

In [2]:
train = pd.read_csv('/Users/hyojin/BOAZ/Base_Analysis/datasets/구내식당/train.csv')
test = pd.read_csv('/Users/hyojin/BOAZ/Base_Analysis/datasets/구내식당/test.csv')

In [3]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


## 메뉴 Split(띄어쓰기, 콤마)

In [4]:
train_lunch = train[['일자','중식메뉴','중식계']]
train_dinner = train[['일자','석식메뉴','석식계']]

In [5]:
def split_menu(x):
  menu_return = []  #return할 메뉴 데이터. 메뉴 스플릿 & 원산지 제외
  for data in x:
    menus = []              #전처리된 메뉴
    menus_sp = data.split() #띄어쓰기 split
    for menu_sp in menus_sp:
      if '(' in menu_sp:
        pass
      elif ',' in menu_sp:  #comma split
        menus_comma = menu_sp.split(',')
        for menu_comma in menus_comma:
          menus.append(menu_comma)
      else:
        menus.append(menu_sp)
    menu_return.append(menus)
  return menu_return

In [6]:
train_lunch['중식메뉴리스트'] = split_menu(train_lunch['중식메뉴'])
train_lunch.head()

/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/2407802181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_lunch['중식메뉴리스트'] = split_menu(train_lunch['중식메뉴'])


,일자,중식메뉴,중식계,중식메뉴리스트
0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...",1039.0,"[쌀밥/잡곡밥, 오징어찌개, 쇠불고기, 계란찜, 청포묵무침, 요구르트, 포기김치]"
1,2016-02-02,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...",867.0,"[쌀밥/잡곡밥, 김치찌개, 가자미튀김, 모둠소세지구이, 마늘쫑무침, 요구르트, 배추..."
2,2016-02-03,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...",1017.0,"[카레덮밥, 팽이장국, 치킨핑거, 쫄면야채무침, 견과류조림, 요구르트, 포기김치]"
3,2016-02-04,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...",978.0,"[쌀밥/잡곡밥, 쇠고기무국, 주꾸미볶음, 부추전, 시금치나물, 요구르트, 포기김치]"
4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...",925.0,"[쌀밥/잡곡밥, 떡국, 돈육씨앗강정, 우엉잡채, 청경채무침, 요구르트, 포기김치]"


### Split한 데이터 확인

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
train_lunch[['일자','중식메뉴리스트']][1000:]

/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/3541698687.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,일자,중식메뉴리스트
1000,2020-03-06,"[곤드레밥, 건새우아욱국, 돈육간장볶음, 쫄면야채무침, 브로컬리, 배추겉절이]"
1001,2020-03-09,"[쌀밥/잡곡밥, 콩나물국, 소불고기, 옥수수계란찜, 오이도라지생채, 포기김치]"
1002,2020-03-10,"[쌀밥/잡곡밥, 대파육개장*칼국수, 가자미엿장조림, 꼬들단무지무침, 청경채겉절이, 포기김치]"
1003,2020-03-11,"[카레라이스, 미소시루, 감자만두*양념, 애기새송이버섯볶음, 골뱅이야채무침, 포기김치]"
1004,2020-03-12,"[쌀밥/잡곡밥, 순두부찌개, 해물돼지갈비찜, 가지완자튀김, 꼬들빼기김치, 열무김치]"
1005,2020-03-13,"[쌀밥/잡곡밥, 설렁탕, 동태포전, 야채스틱, 고추*쌈장, 부추겉절이, 석박지]"
1006,2020-03-16,"[쌀밥/잡곡밥, 어묵국, 돈육굴소스볶음, 오징어숙회무침, 아삭고추쌈장무침, 열무김치]"
1007,2020-03-17,"[쌀밥/잡곡밥, 대구탕, 차돌박이숙주볶음, 브로컬리, 다시마*초장, 시금치나물, 포기김치]"
1008,2020-03-18,"[마파두부덮밥, 조갯살무국, 레몬유린기, 콩나물냉채, 고춧잎볶음, 포기김치]"
1009,2020-03-19,"[쌀밥/잡곡밥, 콩비지찌개, 짜장닭볶음, 햄계란말이, 고추장고구마순무침, 포기김치]"


## 메뉴 종류별 구분(점심)
### 밥(rice), 국(soup), 반찬1(side_1), 반찬2(side_2), 반찬3(side_3),
### 반찬4(side_4), 반찬5(side_5), 디저트1(dessert_1), 디저트2(dessert_2), 김치(kimchi)

In [8]:
def seperate_lunch_menu_list(df):

  str = 'None'
  lunch_menu_list = df['중식메뉴리스트']

  print('######## 점심 데이터 구분 함수 시작 ########')

  #메뉴 종류별 구분 (10 피처)
  rice = []
  soup = []
  side_1 = []
  side_2 = []
  side_3 = []
  side_4 = []
  side_5 = []
  dessert_1 = []
  dessert_2 = []
  kimchi = []

  for idx, lunch_menu in enumerate(lunch_menu_list):

    print("현재 인덱스 :", idx)
    # print(lunch_menu)

    #[밥 국 반찬2개 디저트 김치]
    if idx in [38,57,70,75,80,109,123,286,301,341,
                 361,366,449,482,549,553,687,747,786,843,
                 867,882,909,983,1023,1061]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5])
    #[밥 국 반찬3개 디저트 김치]
    elif idx in [0,1,2,3,4,5,6,7,8,9,
               11,12,13,15,16,17,19,169,378,705,
               951,956,1062,1064,1066]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[5])
      dessert_2.append(str)
      kimchi.append(lunch_menu[6])
    #[밥 국 반찬3개 디저트2개 김치]
    elif idx in [1026,1035,1040,1042,1045]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[5])
      dessert_2.append(lunch_menu[6])
      kimchi.append(lunch_menu[7])       
    #[밥 국 반찬4개 디저트 김치]
    elif idx in [1027,1032,1033,1034,1039,1041,1044,1046]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[5])
      side_5.append(str)
      dessert_1.append(lunch_menu[6])
      dessert_2.append(str)
      kimchi.append(lunch_menu[7])      
    #[밥 국 반찬5개 김치]
    elif idx in [1043]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[5])
      side_5.append(lunch_menu[6])
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[7])        
    #[밥 국 반찬2개 김치]
    elif idx in [223,289,1036]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[4])
    #[밥 국 반찬4개 김치]
    elif idx in [373,873,1007,1029,1030]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[5])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[6])
    #[밥 국 반찬2개 반찬/디저트 김치]
    elif idx in [119,155,189,209,351,471,492,614,648,654,
                 720,742,872,907,914,924,929,937,979,1020,
                 1021]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4].split('/')[0])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4].split('/')[1])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5])
    #[밥 반찬3개 김치]
    elif idx in [175,288]:   
      rice.append(lunch_menu[0])
      soup.append(str)
      side_1.append(lunch_menu[1])
      side_2.append(lunch_menu[2])
      side_3.append(lunch_menu[3])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[4])     
    #[밥 반찬3개 디저트2개 김치]
    elif idx in [1037]:   
      rice.append(lunch_menu[0])
      soup.append(str)
      side_1.append(lunch_menu[1])
      side_2.append(lunch_menu[2])
      side_3.append(lunch_menu[3])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4])
      dessert_2.append(lunch_menu[5])
      kimchi.append(lunch_menu[6])        
    #특이 케이스
    elif idx in [10]:   #[밥 국 반찬2개 디저트(중복2개) 김치]
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4])
      dessert_2.append(str)
      kimchi.append(lunch_menu[6])
    elif idx in [14]:   #[밥 국 반찬2개 디저트2개 김치]
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4])
      dessert_2.append(lunch_menu[5])
      kimchi.append(lunch_menu[6])
    elif idx in [116,400,459]:     #[밥 국 반찬2개 디저트/반찬 김치]
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4].split('/')[1])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4].split('/')[0])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5])  
    elif idx in [412,650]:     #[밥 국 반찬/반찬 반찬 디저트/디저트 김치]
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2].split('/')[0])
      side_2.append(lunch_menu[2].split('/')[1])
      side_3.append(lunch_menu[3])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4].split('/')[0])
      dessert_2.append(lunch_menu[4].split('/')[1])
      kimchi.append(lunch_menu[5])  
    elif idx in [934]:     #[밥 국 반찬2개 디저트/디저트 김치]   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4].split('/')[0])
      dessert_2.append(lunch_menu[4].split('/')[1])
      kimchi.append(lunch_menu[5])  
    #[밥 국 반찬3개 디저트 김치]
    elif idx in [356,840]:     #[<초복특식> 국 밥 반찬 디저트 김치]  
      rice.append(lunch_menu[2])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[3])
      side_2.append(str)
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5])  
    elif idx in [603]:     #[국 밥 반찬2개 디저트 김치]  
      rice.append(lunch_menu[1])
      soup.append(lunch_menu[0])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5])  
    elif idx in [1031]:     #[밥 국 반찬5개 디저트 김치]  
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[5])
      side_5.append(lunch_menu[6])
      dessert_1.append(lunch_menu[7])
      dessert_2.append(str)
      kimchi.append(lunch_menu[8])  
    elif idx in [511,741,980]:     #[밥 국 반찬3개 디저트/김치]  
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[5].split('/')[0])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5].split('/')[1])  
    #이상 케이스
    elif idx in [885]:     #[밥 국 반찬3개 ' /오렌지주스' 김치/디저트]  
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[5].split('/')[1])
      dessert_2.append(lunch_menu[6].split('/')[1])
      kimchi.append(lunch_menu[6].split('/')[0])  
    elif idx in [1038]:     #[밥 국 반찬4개 청,적포도 디저트 김치]  
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[5])
      side_5.append(str)
      dessert_1.append('청,적포도')
      dessert_2.append(lunch_menu[8])
      kimchi.append(lunch_menu[9]) 
    elif idx in [1026]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[5].split(',')[0])
      dessert_2.append(lunch_menu[5].split(',')[0])
      kimchi.append(lunch_menu[6])        
################## 20년 7월 ~ #####################
####  1067 ~  ####
    #[밥 국 반찬3개 김치 반찬]
    elif idx in [1067,1068,1069,1070,1071,1073,1074,1075,1076,1077,
                 1078,1079,1080,1081,1082,1083,1084,1085,1086,1088,
                 1089,1090,1091,1094,1095,1097,1098,1099,1100,1101,
                 1104,1108,1112,1116,1117,1125,1126,1128,1130,1138,
                 1143,1145,1148,1150,1152,1153,1157,1162,1163,1176,
                 1177,1178,1180,1183,1191,1192,1193,1195,1196,1197,
                 1199,1200,1201,1202]:       
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[6])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[5]) 
    #[밥 국 반찬3개 김치 디저트]
    elif idx in [1131]:       
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[6])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5])       
    #[밥 국 반찬3개 김치/디저트 반찬]
    elif idx in [1096,1102,1103,1110,1113,1114,1115,1119,1120,1124,
                 1127,1132,1134,1135,1139,1140,1144,1155,1156,1158,
                 1159,1161,1166,1169,1170,1174,1175,1184,1185,1188,
                 1198,1204]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[6])
      side_5.append(str)
      dessert_1.append(lunch_menu[5].split('/')[1])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5].split('/')[0]) 
    #[밥 국 반찬3개 김치/디저트/디저트 반찬]
    elif idx in [1129]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[6])
      side_5.append(str)
      dessert_1.append(lunch_menu[5].split('/')[1])
      dessert_2.append(lunch_menu[5].split('/')[2])
      kimchi.append(lunch_menu[5].split('/')[0]) 
    #[밥 국 반찬3개 김치 반찬/디저트]
    elif idx in [1107]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[6].split('/')[0])
      side_5.append(str)
      dessert_1.append(lunch_menu[6].split('/')[1])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5]) 
    #[밥 국 반찬3개 김치 디저트/반찬]
    elif idx in [1107]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[6].split('/')[1])
      side_5.append(str)
      dessert_1.append(lunch_menu[6].split('/')[0])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5]) 
    #[밥 국 반찬2개 김치/반찬 반찬]
    elif idx in [1141]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4].split('/')[1])
      side_4.append(lunch_menu[5])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[4].split('/')[0])  
    #[밥 국 반찬3개 김치/반찬 반찬]
    elif idx in [1160,1190,1203]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[5].split('/')[1])
      side_5.append(lunch_menu[6])
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[5].split('/')[0])       
    #[밥 국 반찬3개 김치/반찬]
    elif idx in [94]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[5].split('/')[1])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[5].split('/')[0])       
    #[밥 국 반찬3개 반찬/김치 반찬]
    elif idx in [1189]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[5].split('/')[0])
      side_5.append(lunch_menu[6])
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[5].split('/')[1])    
    #[밥 국 반찬2개 김치/디저트 반찬]
    elif idx in [1109,1149,1164]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[5])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4].split('/')[1])
      dessert_2.append(str)
      kimchi.append(lunch_menu[4].split('/')[0]) 
    #[밥 국 반찬3개 김치/디저트 반찬]
    elif idx in [1092]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[6])
      side_5.append(str)
      dessert_1.append(lunch_menu[5].split('/')[1])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5].split('/')[0])       
    #[밥 국 반찬 반찬/반찬 반찬 김치/디저트 반찬]
    elif idx in [1093]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3].split('/')[0])
      side_3.append(lunch_menu[3].split('/')[1])
      side_4.append(lunch_menu[4])
      side_5.append(lunch_menu[6])
      dessert_1.append(lunch_menu[5].split('/')[1])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5].split('/')[0]) 

    #[밥 국 반찬2개 반찬/김치 반찬]
    elif idx in [1146,1154]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4].split('/')[0])
      side_4.append(lunch_menu[5])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[4].split('/')[1]) 
    #[밥 국 반찬2개 김치 반찬]
    elif idx in [1118,1121,1122,1123,1133,1136,1165,1167,1173,1194]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[5])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[4]) 
    #[밥 국 반찬 김치 반찬]
    elif idx in [1186]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[4])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[3])
    #[밥 국 반찬4개]
    elif idx in [1019,1137]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[5])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(str)
    #[밥 국 반찬3개]
    elif idx in [1151]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(str)
    #[밥 반찬2개 김치/디저트 반찬]
    elif idx in [1111,1179]:   
      rice.append(lunch_menu[0])
      soup.append(str)
      side_1.append(lunch_menu[1])
      side_2.append(lunch_menu[2])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[3].split('/')[1])
      dessert_2.append(str)
      kimchi.append(lunch_menu[3].split('/')[0])    
    elif idx in [1142]:   
      rice.append(lunch_menu[0]+lunch_menu[1])
      soup.append(lunch_menu[2])
      side_1.append(lunch_menu[3])
      side_2.append(lunch_menu[4])
      side_3.append(lunch_menu[5])
      side_4.append(lunch_menu[6].split('/')[1])
      side_5.append(lunch_menu[7])
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[6].split('/')[0]) 
    elif idx in [1087]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[5].split('/')[0])
      side_5.append(lunch_menu[6])
      dessert_1.append(lunch_menu[5].split('/')[2])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5].split('/')[1]) 
    #[밥 반찬3개 김치/디저트 반찬]
    elif idx in [1147]:   
      rice.append(lunch_menu[0])
      soup.append(str)
      side_1.append(lunch_menu[1])
      side_2.append(lunch_menu[2])
      side_3.append(lunch_menu[3])
      side_4.append(lunch_menu[5])
      side_5.append(str)
      dessert_1.append(lunch_menu[4].split('/')[1])
      dessert_2.append(str)
      kimchi.append(lunch_menu[4].split('/')[0]) 
    elif idx in [1168]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4]+' '+lunch_menu[5])
      side_4.append(lunch_menu[7])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[6]) 
    #[국 반찬2개 반찬/김치 반찬]  
    elif idx in [1171]:   
      rice.append(str)
      soup.append(lunch_menu[0])
      side_1.append(lunch_menu[1])
      side_2.append(lunch_menu[2])
      side_3.append(lunch_menu[3].split('/')[0])
      side_4.append(lunch_menu[4])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[3].split('/')[1])  
    #[밥 반찬3개 김치 반찬]  
    elif idx in [1181]:   
      rice.append(lunch_menu[0])
      soup.append(str)
      side_1.append(lunch_menu[1])
      side_2.append(lunch_menu[2])
      side_3.append(lunch_menu[3])
      side_4.append(lunch_menu[5])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[4])     
    #[밥 반찬2개 김치 반찬]  
    elif idx in [1182]:   
      rice.append(lunch_menu[0])
      soup.append(str)
      side_1.append(lunch_menu[1])
      side_2.append(lunch_menu[2])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[3])    
    #[국 반찬3개 김치 반찬]  
    elif idx in [1182,1187]:   
      rice.append(str)
      soup.append(lunch_menu[0])
      side_1.append(lunch_menu[1])
      side_2.append(lunch_menu[2])
      side_3.append(lunch_menu[3])
      side_4.append(lunch_menu[5])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[4])      
    elif idx in [762]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[5])
      dessert_2.append(lunch_menu[6]+'/'+lunch_menu[7])
      kimchi.append(lunch_menu[8])          
##################################################
    else:   #[밥 국 반찬3개 김치]
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[5])

  print('######## 점심 데이터 구분 함수 종료 ########')

  # print(len(rice))
  # print(len(soup))
  # print(len(side_1))
  # print(len(side_2))
  # print(len(side_3))
  # print(len(side_4))
  # print(len(side_5))
  # print(len(dessert_1))
  # print(len(dessert_2))
  # print(len(kimchi))

  return rice, soup, side_1, side_2, side_3, side_4, side_5, dessert_1, dessert_2, kimchi

함수 실행 및 train_lunch 데이터에 추가

In [9]:
rice, soup, side_1, side_2, side_3, side_4, side_5, dessert_1, dessert_2, kimchi = seperate_lunch_menu_list(train_lunch)
# data = seperate_menu_list(train_lunch)

######## 점심 데이터 구분 함수 시작 ########
현재 인덱스 : 0
현재 인덱스 : 1
현재 인덱스 : 2
현재 인덱스 : 3
현재 인덱스 : 4
현재 인덱스 : 5
현재 인덱스 : 6
현재 인덱스 : 7
현재 인덱스 : 8
현재 인덱스 : 9
현재 인덱스 : 10
현재 인덱스 : 11
현재 인덱스 : 12
현재 인덱스 : 13
현재 인덱스 : 14
현재 인덱스 : 15
현재 인덱스 : 16
현재 인덱스 : 17
현재 인덱스 : 18
현재 인덱스 : 19
현재 인덱스 : 20
현재 인덱스 : 21
현재 인덱스 : 22
현재 인덱스 : 23
현재 인덱스 : 24
현재 인덱스 : 25
현재 인덱스 : 26
현재 인덱스 : 27
현재 인덱스 : 28
현재 인덱스 : 29
현재 인덱스 : 30
현재 인덱스 : 31
현재 인덱스 : 32
현재 인덱스 : 33
현재 인덱스 : 34
현재 인덱스 : 35
현재 인덱스 : 36
현재 인덱스 : 37
현재 인덱스 : 38
현재 인덱스 : 39
현재 인덱스 : 40
현재 인덱스 : 41
현재 인덱스 : 42
현재 인덱스 : 43
현재 인덱스 : 44
현재 인덱스 : 45
현재 인덱스 : 46
현재 인덱스 : 47
현재 인덱스 : 48
현재 인덱스 : 49
현재 인덱스 : 50
현재 인덱스 : 51
현재 인덱스 : 52
현재 인덱스 : 53
현재 인덱스 : 54
현재 인덱스 : 55
현재 인덱스 : 56
현재 인덱스 : 57
현재 인덱스 : 58
현재 인덱스 : 59
현재 인덱스 : 60
현재 인덱스 : 61
현재 인덱스 : 62
현재 인덱스 : 63
현재 인덱스 : 64
현재 인덱스 : 65
현재 인덱스 : 66
현재 인덱스 : 67
현재 인덱스 : 68
현재 인덱스 : 69
현재 인덱스 : 70
현재 인덱스 : 71
현재 인덱스 : 72
현재 인덱스 : 73
현재 인덱스 : 74
현재 인덱스 : 75
현재 인덱스 : 76
현재 인덱스 : 77
현재 인덱스 : 78
현재 인덱스 : 79
현재 인덱스 : 80
현재 인

In [10]:
train_lunch.head()

,일자,중식메뉴,중식계,중식메뉴리스트
0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 청포묵무침 요구르트 포기김치 (배추,고추가루:국내산)",1039.0,"[쌀밥/잡곡밥, 오징어찌개, 쇠불고기, 계란찜, 청포묵무침, 요구르트, 포기김치]"
1,2016-02-02,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉절이 (배추,고추가루:국내산)",867.0,"[쌀밥/잡곡밥, 김치찌개, 가자미튀김, 모둠소세지구이, 마늘쫑무침, 요구르트, 배추겉절이]"
2,2016-02-03,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 견과류조림 요구르트 포기김치 (배추,고추가루:국내산)",1017.0,"[카레덮밥, 팽이장국, 치킨핑거, 쫄면야채무침, 견과류조림, 요구르트, 포기김치]"
3,2016-02-04,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 요구르트 포기김치 (배추,고추가루:국내산)",978.0,"[쌀밥/잡곡밥, 쇠고기무국, 주꾸미볶음, 부추전, 시금치나물, 요구르트, 포기김치]"
4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채 청경채무침 요구르트 포기김치 (배추,고추가루:국내산)",925.0,"[쌀밥/잡곡밥, 떡국, 돈육씨앗강정, 우엉잡채, 청경채무침, 요구르트, 포기김치]"


In [11]:
train_lunch['밥'] = rice
train_lunch['국'] = soup
train_lunch['반찬1'] = side_1
train_lunch['반찬2'] = side_2
train_lunch['반찬3'] = side_3
train_lunch['반찬4'] = side_4
train_lunch['반찬5'] = side_5
train_lunch['디저트1'] = dessert_1
train_lunch['디저트2'] = dessert_2
train_lunch['김치'] = kimchi

/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/2263258646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_lunch['밥'] = rice
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/2263258646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_lunch['국'] = soup
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/2263258646.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [12]:
train_lunch.head()

,일자,중식메뉴,중식계,중식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,반찬5,디저트1,디저트2,김치
0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 청포묵무침 요구르트 포기김치 (배추,고추가루:국내산)",1039.0,"[쌀밥/잡곡밥, 오징어찌개, 쇠불고기, 계란찜, 청포묵무침, 요구르트, 포기김치]",쌀밥/잡곡밥,오징어찌개,쇠불고기,계란찜,청포묵무침,None,None,요구르트,None,포기김치
1,2016-02-02,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉절이 (배추,고추가루:국내산)",867.0,"[쌀밥/잡곡밥, 김치찌개, 가자미튀김, 모둠소세지구이, 마늘쫑무침, 요구르트, 배추겉절이]",쌀밥/잡곡밥,김치찌개,가자미튀김,모둠소세지구이,마늘쫑무침,None,None,요구르트,None,배추겉절이
2,2016-02-03,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 견과류조림 요구르트 포기김치 (배추,고추가루:국내산)",1017.0,"[카레덮밥, 팽이장국, 치킨핑거, 쫄면야채무침, 견과류조림, 요구르트, 포기김치]",카레덮밥,팽이장국,치킨핑거,쫄면야채무침,견과류조림,None,None,요구르트,None,포기김치
3,2016-02-04,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 요구르트 포기김치 (배추,고추가루:국내산)",978.0,"[쌀밥/잡곡밥, 쇠고기무국, 주꾸미볶음, 부추전, 시금치나물, 요구르트, 포기김치]",쌀밥/잡곡밥,쇠고기무국,주꾸미볶음,부추전,시금치나물,None,None,요구르트,None,포기김치
4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채 청경채무침 요구르트 포기김치 (배추,고추가루:국내산)",925.0,"[쌀밥/잡곡밥, 떡국, 돈육씨앗강정, 우엉잡채, 청경채무침, 요구르트, 포기김치]",쌀밥/잡곡밥,떡국,돈육씨앗강정,우엉잡채,청경채무침,None,None,요구르트,None,포기김치


### 밥

In [13]:
train_lunch['밥'].unique()

array(['쌀밥/잡곡밥', '카레덮밥', '낙지비빔밥', '쌀밥/오곡밥', '비빔밥', '곤드레밥', '콩나물밥*달래장',
       '봄새싹비빔밥', '콩나물밥', '나물비빔밥', '단호박영양밥', '산채비빔밥', '낙지볶음밥', '열무비빔밥',
       '참치회덮밥', '연복풍덮밥', '열무보리비빔밥', '불고기덮밥', '산채비빔밥*강된장', '짜장밥', '버섯영양밥',
       '소불고기덮밥', '오곡밥', '곤드레밥*강된장', '봄나물비빔밥', '카레라이스', '짜장덮밥', '찹쌀밥',
       '김치제육덮밥', '곤드레밥*간장양념', '단호박카레라이스', '곤드레밥*양념장', '콩나물밥*양념장',
       '마파두부덮밥', '쌀밥/옛날돈까스', '야채볶음밥*짜장소스', '김치볶음밥', '오므라이스', '김치볶음밥/계란',
       '파인애플볶음밥', '김치필라프', '불고기비빔밥', '김밥볶음밥', '고기듬뿍카레라이스', '건강비빔밥*약고추장',
       '새우날치알볶음밥', '소고기콩나물밥*양념', '닭살카레라이스', '열무보리비빔밥/강된장', '베이컨김치볶음밥',
       '돈육버섯고추장덮밥', '비빔밥*약고추장', '쌀밥/흑미밥/찰현미밥', '쌀밥/치자밥/찰현미밥',
       '쌀밥/병아리콩밥/찰현미밥', '쌀밥/기장밥/찰현미밥', '낙지비빔밥*계란후라이', '쌀밥/수수밥/찰현미밥',
       '쌀밥/검정콩밥/찰현미밥', '쌀밥/렌틸콩밥/찰현미밥', '쌀밥/차조밥/찰현미밥', '쌀밥/보리밥/찰현미밥',
       '쌀밥/찰보리밥/찰현미밥', '계란채소볶음밥', '쌀밥/귀리밥/찰현미밥', '쌀밥/병아리콩/찰현미밥',
       '쌀밥/곤드레밥/찰현미밥', '전주비빔밥', '쌀밥/완두콩밥/찰현미밥', '쌀밥/강낭콩밥/찰현미밥', 'None',
       '쌀밥/팥밥/찰현미밥', '전주비빔밥*약고추장'], dtype=object)

### 국

In [14]:
train_lunch['국'].unique()

array(['오징어찌개', '김치찌개', '팽이장국', '쇠고기무국', '떡국', '시래기국', '꽃게탕', '시금치국',
       '냉이된장국', '유부장국', '대구찌개', '쇠고기느타리국', '콩나물국', '북어계란국', '수제비국',
       '얼갈이된장국', '쇠고기미역국', '근대된장국', '부대찌개', '차돌박이찌개', '북어국', '고추장찌개',
       '배추된장국', '어묵국', '쑥국', '콩나물김치국', '사골우거지국', '쇠고기샤브국', '맑은국', '닭개장',
       '옹심이만두국', '쪽파국', '매운콩나물국', '버섯들깨탕', '순두부찌개', '올갱이아욱국', '냉이된장찌개',
       '육개장', '대구매운탕', '미소장국', '미역국', '돈육김치찌개', '무채국', '매운어묵국', '시래기된장국',
       '오징어국', '된장찌개', '감자고추장찌개', '들깨버섯탕', '열무된장국', '북어콩나물국', '한방갈비탕',
       '미역장국', '쇠고기해장국', '강된장찌개', '오이미역냉국', '참치김치찌개', '도토리묵냉국', '버섯들깨찌개',
       '가쯔오장국', '우무묵냉국', '호박잎찌개', '건새우아욱국', '오이냉국', '사골우거지탕', '우묵콩국',
       '청국장찌개', '부대찌개*라면사리', '복숭아미역냉국', '호박잎된장국', '사과오이냉국', '닭곰탕',
       '해물된장찌개', '버섯매운탕', '두부맑은국', '황태맑은국', '토란국', '아욱된장국', '느타리버섯국',
       '우거지해장국', '시금치된장국', '팽이버섯국', '건새우무국', 'None', '계란국', '동태찌개', '알탕',
       '김치콩나물국', '김치국', '조랭이떡미역국', '설렁탕', '콩비지찌개', '순두부탕', '근대국', '홍합탕',
       '아욱국', '차돌박이된장찌개', '얼갈이국', '사골떡국', '황태국', '짬뽕국', '냉이콩나물국',
       '

### 반찬1

In [15]:
train_lunch['반찬1'].unique()

array(['쇠불고기', '가자미튀김', '치킨핑거', '주꾸미볶음', '돈육씨앗강정', '훈제오리구이', '돈육굴소스볶음',
       '닭감자조림', '탕수어', '쇠고기장조림', '고구마치즈구이', '마늘간장치킨', '갈치구이', '돈육간장볶음',
       '소세지피망볶음', '닭갈비', '오징어볶음', '제육볶음', '탕수육', '고등어김치말이', '돈육장조림',
       '닭데리야끼조림', '수제돈가스', '돈육고추장볶음', '소불고기', '미니채소떡갈비', '버섯불고기', '코다리강정',
       '치킨텐더*요거트D', '자반고등어구이', '오징어튀김', '갈치조림', '오징어볶음*소면', '매운돼지갈비찜',
       '순살양념치킨', '쇠고기단호박조림', '돈육강정', '꽁치한마리구이', '매콤떡갈비조림', '쇠고기볶음',
       '등갈비김치찜', '돈간장불고기', '궁중떡찜', '양념파닭', '미트볼조림', '모둠장조림', '돈육칠리강정',
       '찜닭', '닭오븐구이', '쇠고기불고기', '꼬지삼색전', '닭양념조림', '누룽지탕수육', '순살깐풍기',
       '돼지갈비찜', '떡갈비조림', '가자미양념찜', '레몬탕수육', '간장치킨', '돈삼겹보쌈', '매콤낙지볶음',
       '목살데리야끼구이', '고등어구이', '간장돼지갈비찜', '유자청제육볶음', '꽁치김치말이찜', '닭강정',
       '갈릭돈까스', '순살파닭', '사천식탕수육', '매운주꾸미볶음', '돈육떡강정', '목살데리야끼', '고추장불고기',
       '불닭볶음', '동파삼겹수육', '훈제오리냉채', '꽁치오븐구이', '꽃맛살볶음', '매콤돼지갈비찜', '미니함박조림',
       '장어강정', '삼겹살오븐구이', '새우까스&탈탈소스', '고등어조림', '갈릭돈가스', '소고기숙주볶음',
       '고추잡채*꽃빵', '칠리탕수육', '닭볶음', '동태무조림', '마늘치킨', '계란말이', '닭찜',
       '가자미카

### 반찬2

In [16]:
train_lunch['반찬2'].unique()

array(['계란찜', '모둠소세지구이', '쫄면야채무침', '부추전', '우엉잡채', '도토리묵무침', '옥수수전',
       '연두부*양념장', '오징어숙회무침', '통도라지구이', '해초무침', '새송이버섯볶음', '호박꼬지&고사리&무나물',
       '버섯잡채', '맛탕', '새싹두부구이', '늙은호박전', '맛살계란말이', '새송이버섯조림', '메추리알조림',
       '모둠묵*양념장', '소세지구이', '홍어무침', '해물파전', '탕평채', '모둠양채쌈*쌈장', '골뱅이무침*소면',
       '두부계란부침', '파래김*양념장', '연근땅콩조림', '계란말이', '두부&김치볶음', '땅콩조림', '오징어초무침',
       '비엔나볶음', '해파리냉채', '새송이버섯전', '알감자버터구이', '동태전', '브로컬리맛살볶음',
       '모둠소시지구이', '두부조림', '무쌈/양파절임', '열무나물', '양배추쌈', '도라지초무침', '콩나물잡채',
       '고기전', '청포묵무침', '연근조림', '김치전', '치즈계란말이', '동그랑땡부침', '고등어카레구이',
       '맛살겨자채', '온두부', '멸치크랜베리볶음', '꽃맛살볶음', '우엉조림', '비엔나감자볶음', '동그랑땡전',
       '명태조림', '잡채', '궁중떡볶이', '두부구이', '홍어채무침', '맛살콩나물냉채', '참치야채전', '쫄면무침',
       '명태엿장조림', '마늘쫑볶음', '골뱅이무침&소면', '실곤약초무침', '닭가슴살냉채', '두부새싹구이',
       '맛살겨자초무침', '돈육잡채', '감자볶음', '감자만두', '삼치구이', '춘권', '해파리겨자채', '감자채볶음',
       '마늘쫑무침', '닭살겨자냉채', '웨지감자오븐구이', '연근/오징어튀김', '오꼬노미계란말이', '오징어브로컬리숙회',
       '매운어묵볶음', '고구마범벅', '비엔나피망볶음', '꽁치와사비구이', '모듬소시지볶음', '과일샐러드',

### 반찬3

In [17]:
train_lunch['반찬3'].unique()

array(['청포묵무침', '마늘쫑무침', '견과류조림', '시금치나물', '청경채무침', '쌈무/양파절임', '유채나물',
       '콩나물무침', '취나물', '치커리무침', 'None', '돌나물무침/치킨무', '파래김*양념장', '꽃상추무침',
       '곰피초장', '도라지초무침', '치커리생채', '아삭고추무침', '무생채', '열무된장나물', '톳두부무침',
       '풋마늘초무침', '쫄면무침', '콩나물파채무침', '마늘쫑볶음', '무나물', '깻순나물', '쌈무/부추생채',
       '도토리묵무침', '실곤약초무침', '시래기나물', '열무나물', '치커리유자청생채', '봄동나물', '모둠쌈*쌈장',
       '브로컬리두부무침', '해초무침', '부추생채', '청경채생채', '깻잎양념지', '버섯볶음', '풋마늘대무침',
       '콩자반/치킨무', '치커리사과무침', '비름나물', '가지나물', '통도라지구이', '치커리유자생채', '숙주나물',
       '불미나리무침', '오이무침', '고구마순나물', '오이도라지생채', '머위대나물', '브로컬리초장', '모둠쌈',
       '어묵볶음', '김구이', '오이무침/치킨무', '간장깻잎지', '쌈무/야채스틱', '상추겉절이', '양배추쌈*쌈장',
       '노각무침', '오이생채', '영양부추', '모듬쌈*쌈장', '도라지나물', '치커리들깨무침', '깻잎지', '락교',
       '쌈무/참나물생채', '연근조림', '오이도라지무침', '콩자반', '양배추피클', '꽈리고추찜', '시금치무침',
       '상추&마늘', '쑥갓두부무침', '치커리유자청무침', '올방개묵무침', '꽃맛살오리엔탈샐러드', '호박잎쌈*양념',
       '단무지무침', '연두부*양념', '참나물땅콩무침', '애호박나물', '참나물생채', '핫도그', '단배추나물',
       '양배추쌈&쌈장', '마늘쫑건새우볶음', '동태/새송이전', '참나물무침', '오복지무침', '

### 반찬4

In [18]:
train_lunch['반찬4'].unique()

array(['None', '상추쌈', '*쌈장', '부추겉절이', '시금치나물', '참치김치볶음', '참나물땅콩가루무침',
       '비트무피클', '오이사과생채', '양배추', '가지나물무침', '두릅새송이초무침', '문어오이미역초무침',
       '콩나물부추무침', '황도샐러드', '치커리유자샐러드', '호박숙*양념장', '인절미츄러스맛탕', '해초샐러드',
       '양상추샐러드*딸기요거트', '양상추샐러드*오리엔탈', '요구르트', '양상추샐러드*흑임자드레싱',
       '그린샐러드*오렌지드레싱/핫도그', '시저샐러드*시저드레싱', '양상추샐러드/마카로니샐러드', '양상추샐러드*요거트D',
       '비타민흑임자샐러드*흑임자D', '치커리만다린샐러드*오렌지D', '양상추샐러드*망고D', '양배추샐러드*사우전D',
       '그린샐러드*키위D/타래과', '로스트치킨샐러드*오리엔탈D', '새싹샐러드*오렌지D', '양배추샐러드*요플레D',
       '양상추샐러드*유자청D', '토마토샐러드*발사믹D', '부추무침', '그린샐러드*매실D', '양배추샐러드*딸기요거트D',
       '양상추샐러드*사우전D', '견과류샐러드*땅콩D', '눈꽃치즈샐러드*오리엔탈D', '파프리카해초무침',
       '시리얼샐러드*요플레D', '양상추샐러드*파인애플D', '푸실리파스타샐러드', '그린샐러드*오리엔탈D',
       '황도샐러드*밀감D', '사과푸딩샐러드*요플레D', '나쵸콥샐러드*토마토살사소스', '그린샐러드*참깨D',
       '마카로니콘샐러드', '크루통샐러드*참깨D', '양상추샐러드*매실D', '모닝샌드', '견과류샐러드*딸기D',
       '황도샐러드*황도D', '그린샐러드*요거트D', '양상추샐러드*딸기D', '양상추샐러드*키위D', '그린샐러드*살구D',
       '양상추샐러드*오리엔탈D', '과일샐러드', '양상추샐러드*오미자D', '시금치샐러드*오리엔탈D',
       '시리얼샐러드*요거트D', '크루통샐러드*

### 반찬5

In [19]:
train_lunch['반찬5'].unique()

array(['None', '케일*우렁쌈장', '꽃맛살과일샐러드', '그린샐러드*참깨D', '콜리샐러드*발사믹D',
       '견과류샐러드*요거트D', '양상추샐러드*딸기D', '양상추샐러드*키위D', '양상추샐러드*오리엔탈D',
       '양상추샐러드*오렌지D'], dtype=object)

### 디저트1

In [20]:
train_lunch['디저트1'].unique()

array(['요구르트', '부럼/요구르트', '귤', 'None', '오렌지', '유기농식혜', '요플레', '매실주스',
       '음료', '수박', '토마토', '오렌지주스', '아이스슈', '바나나', '사과', '천도복숭아',
       '샐러드바:과일', '미니케익', '군고구마', '식혜', '부럼', '과일', '송편', '크리스마스케익',
       '방울토마토', '송편/식혜/과일', '조각케익', '살구복숭아주스', '짜요짜요', '고구마치즈빵', '청포도',
       '오미자주스', '자몽에이드', '요거트드링킹', '홍삼', '망고', '청,적포도', '포도주스', '알로에주스',
       '배즙', '파인애플', '사과즙', '수박화채', '탱크보이', '찐옥수수', '수떡수떡화채', '츄러스',
       '우무콩국', '딸기푸딩', '나쵸칩', '열대과일', '사과나무주스', '햇고구마오븐구이', '요거트파르페',
       '인절미', '찹쌀호떡', '배도라지주스', '푸딩', '단감', '고구마오븐구이', '꽈배기도넛', '나쵸',
       '라면땅', '홍시', '수제과일잼샌드', '붕어빵', '오프룻요거트', '꿀호떡', '청포도주스', '새알팥죽',
       '마늘바게트'], dtype=object)

### 디저트2

In [21]:
train_lunch['디저트2'].unique()

array(['None', '요구르트', '식혜', '오렌지/바나나', '과일', '오렌지', '크란치바', '복숭아아이스티',
       '오렌지주스', '프로바이오틱', '마시는요거트', '요플레'], dtype=object)

### 김치

In [22]:
train_lunch['김치'].unique()

array(['포기김치', '배추겉절이', '깍두기', '열무김치', '볶은김치', '석박지', '알타리김치', '파김치',
       '갓김치', '베추겉절이', '얼갈이겉절이', '오이소박이', '오복지무침', '볶음김치', '부추겉절이',
       '수제석박지', '김치볶음', 'None', '배추김치', '백김치', '오이소배기', '보쌈김치', '겉절이김치',
       '총각김치', '쌈추겉절이', '무생채', '봄동겉절이', '단호박물김치'], dtype=object)

In [23]:
train_lunch['디저트1'].value_counts()

None        1057
요구르트        25  
요플레         11  
수박          7   
식혜          6   
바나나         6   
과일          6   
귤           6   
음료          5   
오렌지         4   
군고구마        3   
사과          3   
부럼          3   
유기농식혜       2   
오렌지주스       2   
딸기푸딩        2   
나쵸칩         2   
방울토마토       2   
사과나무주스      1   
인절미         1   
요거트파르페      1   
햇고구마오븐구이    1   
새알팥죽        1   
열대과일        1   
배도라지주스      1   
우무콩국        1   
츄러스         1   
수떡수떡화채      1   
찹쌀호떡        1   
나쵸          1   
푸딩          1   
단감          1   
고구마오븐구이     1   
꽈배기도넛       1   
라면땅         1   
홍시          1   
탱크보이        1   
수제과일잼샌드     1   
붕어빵         1   
오프룻요거트      1   
꿀호떡         1   
청포도주스       1   
찐옥수수        1   
망고          1   
수박화채        1   
짜요짜요        1   
매실주스        1   
토마토         1   
아이스슈        1   
천도복숭아       1   
샐러드바:과일     1   
미니케익        1   
송편          1   
크리스마스케익     1   
송편/식혜/과일    1   
조각케익        1   
살구복숭아주스     1   
고구마치즈빵      1   
사과즙         1 

## 메뉴 종류별 구분(저녁)
### 밥(rice), 국(soup), 반찬1(side_1), 반찬2(side_2), 반찬3(side_3),
### 반찬4(side_4), 디저트1(dessert_1), 디저트2(dessert_2), 김치(kimchi)

In [24]:
train_dinner['석식메뉴리스트'] = split_menu(train_dinner['석식메뉴'])
train_dinner.head()

/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/3258645492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dinner['석식메뉴리스트'] = split_menu(train_dinner['석식메뉴'])


,일자,석식메뉴,석식계,석식메뉴리스트
0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 포기김치 (김치:국내산)",331.0,"[쌀밥/잡곡밥, 육개장, 자반고등어구이, 두부조림, 건파래무침, 포기김치]"
1,2016-02-02,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무침 바나나 포기김치 (배추,고추가루:국내산)",560.0,"[콩나물밥*양념장, 어묵국, 유산슬, 아삭고추무침, 바나나, 포기김치]"
2,2016-02-03,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기전 (돼지고기:국내산) 새송이버섯볶음 포기김치 (배추,고추가루:국내산)",573.0,"[쌀밥/잡곡밥, 청국장찌개, 황태양념구이, 고기전, 새송이버섯볶음, 포기김치]"
3,2016-02-04,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포기김치 (배추,고추가루:국내산)",525.0,"[미니김밥*겨자장, 우동, 멕시칸샐러드, 군고구마, 무피클, 포기김치]"
4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:국내산) 감자소세지볶음 콩나물무침 포기김치 (배추,고추가루:국내산)",330.0,"[쌀밥/잡곡밥, 차돌박이찌개, 닭갈비, 감자소세지볶음, 콩나물무침, 포기김치]"


In [25]:
train_dinner[['일자','석식메뉴리스트']].loc[93]

일자         2016-06-21                           
석식메뉴리스트    [김밥/맑은국, 비빔국수, 콘샐러드, 찐햇감자, 단무지, 포기김치]
Name: 93, dtype: object

In [26]:
def seperate_dinner_menu_list(df):

  str = 'None'
  dinner_menu_list = df['석식메뉴리스트']

  print('######## 저녁 데이터 구분 함수 시작 ########')

  #메뉴 종류별 구분 (10 피처)
  rice = []
  soup = []
  side_1 = []
  side_2 = []
  side_3 = []
  side_4 = []
  side_5 = []
  dessert_1 = []
  dessert_2 = []
  kimchi = []

  for idx, dinner_menu in enumerate(dinner_menu_list):
    print("현재 인덱스 :", idx)
    # print(dinner_menu)

    #[밥 국 반찬2개 디저트 김치]
    if idx in [1,9,11,34,37,47,48,64,66,71,74,81,85,88,100,124,
               134,144,154,170,173,188,203,220,228,230,238,240,
               253,258,261,287,302,321,332,340,352,355,367,381,
               386,404,435,455,473,481,486,488,493,496,513,523,
               525,535,548,550,570,580,588,593,600,608,610,624,
               629,639,647,653,656,678,683,686,693,701,726,729,
               741,760,777,782,819,832,834,842,847,852,859,873,
               901,906,921,946,952,966,1017,1042,1055,1065,110]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[4])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])
    #[밥 국 반찬2개 반찬/디저트 김치]
    elif idx in [29,103,105,120,137,178,193,201,213,223,251,
                 256,280,285,296,297,300,305,310,318,320,328,342,
                 384,399,407,411,416,443,456,458,474,509,516,536,
                 540,545,563,590,605,632,642,661,663,703,716,719,
                 731,736,738,743,750,752,755,757,770,775,787,795,
                 798,803,808,813,822,854,866,876,886,891,913,943,
                 948,984,1009,1014,1155]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4].split('/')[0])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[4].split('/')[1])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])    
    #[밥 국 반찬2개 디저트/반찬 김치]
    elif idx in [53,56,98,190,233,362,530,561,567,572,765,767,898,326]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4].split('/')[1])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[4].split('/')[0])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5]) 
    #[밥 국 반찬2개 디저트/디저트 김치]
    elif idx in [62,69,113,132,246,267,483,964,1063]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[4].split('/')[0])
      dessert_2.append(dinner_menu[4].split('/')[1])
      kimchi.append(dinner_menu[5]) 
    #[밥 국 반찬2개 김치]
    elif idx in [214,273,1087,1094,1115,1119,1124,1127,1130,1131,1138,
                 1145,1148,1152,1156,1169,1180,1182,1191,565]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[4]) 
    #[밥 국 반찬2개 김치/반찬]
    elif idx in [1143]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4].split('/')[1])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[4].split('/')[0]) 
    #[밥 국 반찬3개 디저트 김치]
    elif idx in [225,373,620,627,951,1038,1039,1041]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[5])
      dessert_2.append(str)
      kimchi.append(dinner_menu[6]) 
    #[밥 국 반찬3개 김치/반찬]
    elif idx in [49,1141]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4])
      side_4.append(dinner_menu[5].split('/')[1])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[5].split('/')[0]) 
    #[밥 국 반찬3개 김치/디저트]
    elif idx in [1105,1111,1117]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[5].split('/')[1])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5].split('/')[0])
    #[밥 국 반찬4개 김치]
    elif idx in [331,394,396,1031,1032,1035,1037,1043,1068]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4])
      side_4.append(dinner_menu[5])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[6])
    #[밥 국 반찬4개 디저트 김치]
    elif idx in [971]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4])
      side_4.append(dinner_menu[5])
      side_5.append(str)
      dessert_1.append(dinner_menu[6])
      dessert_2.append(str)
      kimchi.append(dinner_menu[7])
    #[밥 반찬 국 반찬 디저트 김치]
    elif idx in [58]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[2])
      side_1.append(dinner_menu[1])
      side_2.append(dinner_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[4])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])
    #[밥 국 반찬 반찬/디저트 반찬 김치]
    elif idx in [128,130,208]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3].split('/')[0])
      side_3.append(dinner_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[3].split('/')[1])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])
    #[밥 국 반찬/디저트 반찬2개 김치]
    elif idx in [185]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2].split('/')[0])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[2].split('/')[1])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])
    #[밥 국 반찬 디저트/디저트 반찬 김치]
    elif idx in [215]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[4])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[3].split('/')[0])
      dessert_2.append(dinner_menu[3].split('/')[1])
      kimchi.append(dinner_menu[5])
    #[밥 국 반찬 디저트 디저트/반찬 김치]
    elif idx in [270]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[4].split('/')[1])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[3])
      dessert_2.append(dinner_menu[4].split('/')[0])
      kimchi.append(dinner_menu[5])
    #[밥/국 반찬4개 김치]
    elif idx in [93]:   
      rice.append(dinner_menu[0].split('/')[0])
      soup.append(dinner_menu[0].split('/')[1])
      side_1.append(dinner_menu[1])
      side_2.append(dinner_menu[2])
      side_3.append(dinner_menu[3])
      side_4.append(dinner_menu[4])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])
    #[국 밥 반찬2개 디저트 김치]
    elif idx in [345]:   
      rice.append(dinner_menu[1])
      soup.append(dinner_menu[0])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[4])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])
    #[밥 반찬 국 반찬2개 디저트 김치]
    elif idx in [391]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[2])
      side_1.append(dinner_menu[1])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[5])
      dessert_2.append(str)
      kimchi.append(dinner_menu[6])
    #[밥 국 반찬 김치]
    elif idx in [1172]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(str)
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[3])
    #[밥 국 반찬 디저트 김치]
    elif idx in [528,790]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(str)
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[3])
      dessert_2.append(str)
      kimchi.append(dinner_menu[4])
    #[밥 국 반찬 디저트 디저트 김치]
    elif idx in [1013]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(str)
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[3])
      dessert_2.append(dinner_menu[4])
      kimchi.append(dinner_menu[5])
    #[밥 국 반찬 디저트 반찬 김치]
    elif idx in [44,151,161,335,453,3,19]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[4])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[3])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])
    #[밥 국 반찬 x 반찬2개 김치]
    elif idx in [15]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[4])
      side_3.append(dinner_menu[5])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[6])
    #[]
    elif idx in [95]:   
      rice.append(dinner_menu[1])
      soup.append(dinner_menu[2])
      side_1.append(dinner_menu[3])
      side_2.append(dinner_menu[4].split('/')[0])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[4].split('/')[1])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])
    #[밥 반찬2개 디저트/반찬 김치]
    elif idx in [506]:   
      rice.append(dinner_menu[0])
      soup.append(str)
      side_1.append(dinner_menu[1])
      side_2.append(dinner_menu[2])
      side_3.append(dinner_menu[3].split('/')[1])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[3].split('/')[0])
      dessert_2.append(str)
      kimchi.append(dinner_menu[4])
    #[국 밥 반찬 반찬/디저트 김치]
    elif idx in [780]:   
      rice.append(dinner_menu[1])
      soup.append(dinner_menu[0])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3].split('/')[0])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[3].split('/')[1])
      dessert_2.append(str)
      kimchi.append(dinner_menu[4])
    #[]
    elif idx in [1010]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[5])
      side_3.append(dinner_menu[6])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[7])
    #[]
    elif idx in [1027]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[3])
      side_1.append(dinner_menu[1]+ ' '+dinner_menu[2])
      side_2.append(dinner_menu[4])
      side_3.append(dinner_menu[5])
      side_4.append(dinner_menu[6])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[7])
    #[]
    elif idx in [1048]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append('배')
      dessert_2.append(dinner_menu[5])
      kimchi.append(dinner_menu[6])
    #[]
    elif idx in [1086]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[2])
      side_1.append(dinner_menu[3].split('/')[1])
      side_2.append(dinner_menu[4].split('/')[0])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[3].split('/')[0])
      dessert_2.append(dinner_menu[4].split('/')[1])
      kimchi.append(dinner_menu[5])
    #[국 반찬3개 김치]
    elif idx in [1137,1140]:   
      rice.append(str)
      soup.append(dinner_menu[0])
      side_1.append(dinner_menu[1])
      side_2.append(dinner_menu[2])
      side_3.append(dinner_menu[3])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[4])
    #[밥 국 반찬/반찬 반찬2개]
    elif idx in [1167]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2].split('/')[0])
      side_2.append(dinner_menu[2].split('/')[1])
      side_3.append(dinner_menu[3])
      side_4.append(dinner_menu[4])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(str)
    #[국 반찬2개 김치]
    elif idx in [1184]:   
      rice.append(str)
      soup.append(dinner_menu[0])
      side_1.append(dinner_menu[1])
      side_2.append(dinner_menu[2])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[3])
    #[밥 반찬3개 김치]
    elif idx in [1187]:   
      rice.append(dinner_menu[0])
      soup.append(str)
      side_1.append(dinner_menu[1])
      side_2.append(dinner_menu[2])
      side_3.append(dinner_menu[3])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[4])
    #[]
    elif idx in [1189]:   
      rice.append(dinner_menu[0])
      soup.append(str)
      side_1.append(dinner_menu[1])
      side_2.append(dinner_menu[2])
      side_3.append(dinner_menu[3])
      side_4.append(dinner_menu[4])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(str)
    #[]
    elif idx in [1204]:   
      rice.append(dinner_menu[0])
      soup.append(str)
      side_1.append(dinner_menu[1])
      side_2.append(dinner_menu[2])
      side_3.append(dinner_menu[3])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[4])
    #[석식 없는 날]
    elif idx in [204,224,244,262,281,306,392,412,424,449,468,
                 492,510,529,549,571,586,589,609,633,648,687,
                 872,890,912,932,955,
                 327,346,667,706,730,747,766,786,809,828,853,
                 973,993,1166,
                 366]:   
      rice.append('No dinner')
      soup.append(str)
      side_1.append(str)
      side_2.append(str)
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(str)

    #[밥 국 반찬3개 김치]
    else:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])

  print('######## 저녁 데이터 구분 함수 종료 ########')

  # print(len(rice))
  # print(len(soup))
  # print(len(side_1))
  # print(len(side_2))
  # print(len(side_3))
  # print(len(side_4))
  # print(len(side_5))
  # print(len(dessert_1))
  # print(len(dessert_2))
  # print(len(kimchi))

  return rice, soup, side_1, side_2, side_3, side_4, dessert_1, dessert_2, kimchi

In [27]:
rice, soup, side_1, side_2, side_3, side_4, dessert_1, dessert_2, kimchi = seperate_dinner_menu_list(train_dinner)

######## 저녁 데이터 구분 함수 시작 ########
현재 인덱스 : 0
현재 인덱스 : 1
현재 인덱스 : 2
현재 인덱스 : 3
현재 인덱스 : 4
현재 인덱스 : 5
현재 인덱스 : 6
현재 인덱스 : 7
현재 인덱스 : 8
현재 인덱스 : 9
현재 인덱스 : 10
현재 인덱스 : 11
현재 인덱스 : 12
현재 인덱스 : 13
현재 인덱스 : 14
현재 인덱스 : 15
현재 인덱스 : 16
현재 인덱스 : 17
현재 인덱스 : 18
현재 인덱스 : 19
현재 인덱스 : 20
현재 인덱스 : 21
현재 인덱스 : 22
현재 인덱스 : 23
현재 인덱스 : 24
현재 인덱스 : 25
현재 인덱스 : 26
현재 인덱스 : 27
현재 인덱스 : 28
현재 인덱스 : 29
현재 인덱스 : 30
현재 인덱스 : 31
현재 인덱스 : 32
현재 인덱스 : 33
현재 인덱스 : 34
현재 인덱스 : 35
현재 인덱스 : 36
현재 인덱스 : 37
현재 인덱스 : 38
현재 인덱스 : 39
현재 인덱스 : 40
현재 인덱스 : 41
현재 인덱스 : 42
현재 인덱스 : 43
현재 인덱스 : 44
현재 인덱스 : 45
현재 인덱스 : 46
현재 인덱스 : 47
현재 인덱스 : 48
현재 인덱스 : 49
현재 인덱스 : 50
현재 인덱스 : 51
현재 인덱스 : 52
현재 인덱스 : 53
현재 인덱스 : 54
현재 인덱스 : 55
현재 인덱스 : 56
현재 인덱스 : 57
현재 인덱스 : 58
현재 인덱스 : 59
현재 인덱스 : 60
현재 인덱스 : 61
현재 인덱스 : 62
현재 인덱스 : 63
현재 인덱스 : 64
현재 인덱스 : 65
현재 인덱스 : 66
현재 인덱스 : 67
현재 인덱스 : 68
현재 인덱스 : 69
현재 인덱스 : 70
현재 인덱스 : 71
현재 인덱스 : 72
현재 인덱스 : 73
현재 인덱스 : 74
현재 인덱스 : 75
현재 인덱스 : 76
현재 인덱스 : 77
현재 인덱스 : 78
현재 인덱스 : 79
현재 인덱스 : 80
현재 인

In [28]:
train_dinner['밥'] = rice
train_dinner['국'] = soup
train_dinner['반찬1'] = side_1
train_dinner['반찬2'] = side_2
train_dinner['반찬3'] = side_3
train_dinner['반찬4'] = side_4
train_dinner['디저트1'] = dessert_1
train_dinner['디저트2'] = dessert_2
train_dinner['김치'] = kimchi

/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/738244941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dinner['밥'] = rice
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/738244941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dinner['국'] = soup
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/738244941.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [29]:
train_dinner.head()

,일자,석식메뉴,석식계,석식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,디저트1,디저트2,김치
0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 포기김치 (김치:국내산)",331.0,"[쌀밥/잡곡밥, 육개장, 자반고등어구이, 두부조림, 건파래무침, 포기김치]",쌀밥/잡곡밥,육개장,자반고등어구이,두부조림,건파래무침,None,None,None,포기김치
1,2016-02-02,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무침 바나나 포기김치 (배추,고추가루:국내산)",560.0,"[콩나물밥*양념장, 어묵국, 유산슬, 아삭고추무침, 바나나, 포기김치]",콩나물밥*양념장,어묵국,유산슬,아삭고추무침,None,None,바나나,None,포기김치
2,2016-02-03,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기전 (돼지고기:국내산) 새송이버섯볶음 포기김치 (배추,고추가루:국내산)",573.0,"[쌀밥/잡곡밥, 청국장찌개, 황태양념구이, 고기전, 새송이버섯볶음, 포기김치]",쌀밥/잡곡밥,청국장찌개,황태양념구이,고기전,새송이버섯볶음,None,None,None,포기김치
3,2016-02-04,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포기김치 (배추,고추가루:국내산)",525.0,"[미니김밥*겨자장, 우동, 멕시칸샐러드, 군고구마, 무피클, 포기김치]",미니김밥*겨자장,우동,멕시칸샐러드,무피클,None,None,군고구마,None,포기김치
4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:국내산) 감자소세지볶음 콩나물무침 포기김치 (배추,고추가루:국내산)",330.0,"[쌀밥/잡곡밥, 차돌박이찌개, 닭갈비, 감자소세지볶음, 콩나물무침, 포기김치]",쌀밥/잡곡밥,차돌박이찌개,닭갈비,감자소세지볶음,콩나물무침,None,None,None,포기김치


In [30]:
train_dinner['밥'].unique()

array(['쌀밥/잡곡밥', '콩나물밥*양념장', '미니김밥*겨자장', '참치회덮밥', '볶음밥*자장소스', '김밥',
       '멸치주먹밥', '카레덮밥', '볶음밥', '짜장잡채덮밥', '토마토스파게티', '굴소스파인볶음밥', '햄볶음밥',
       '주먹밥', '김치볶음밥', '치킨마요덮밥', '비빔밥', '낙지비빔밥', '오므라이스', '짜장덮밥', '참치주먹밥',
       '김치볶음밥/계란', '돈까스김밥', '유부초밥', '새우살볶음밥', '짜장잡채밥', '날치알볶음밥', '충무김밥',
       '볶음밥/짜장소스', '콩나물밥', '카레라이스', '작은밥', '김밥볶음밥', '콩국수', '굴소스해물볶음밥',
       '쇠고기숙주규동덮밥', '통새우김밥', '나가사키면&김', '옛날왕돈까스', '낙지볶음덮밥', '생선초밥',
       '후난식볶음밥', 'No dinner', '해물굴소스볶음밥', '마파두부덮밥', '김치참치주먹밥', '쇠고기덮밥',
       '날치알김치볶음밥', '참치야채비빔밥', '베이컨김치볶음밥', '해물볶음밥', '나가사끼짬뽕', '닭갈비볶음밥',
       '야채볶음밥', '꼬마김밥*겨자장', '소고기주먹밥', '빠에야', '목살필라프', '김치주먹밥', '산채비빔밥',
       '낙지덮밥', '볶음밥*짜장소스', '김치우동', '볶음밥&짜장소스', '하와이안필라프', '야채주먹밥',
       '곤드레밥*강된장', '투움바스파게티', '짜장밥', '새우볶음밥', '쌀밥/옛날돈까스', '잡채밥&짜장소스',
       '미니햄버거', '파인애플볶음밥', '삼겹살볶음밥', '잡채밥/짜장소스', '베이컨볶음밥', '중국식볶음밥',
       '제육김치덮밥', '김치필라프', '참치덮밥', '쌀밥', '짜장면/작은밥', '참치마요덮밥', '굴소스볶음밥',
       '쇠고기볶음밥', '스팸주먹밥', '유산슬덮밥', '버섯영양밥*우렁양념장', '찹쌀밥', '마늘베이컨볶음밥',
       '스팸볶음

In [31]:
train_dinner['국'].unique()

array(['육개장', '어묵국', '청국장찌개', '우동', '차돌박이찌개', '맑은국', '김치콩나물국', '홍합미역국',
       '된장찌개', '순두부찌개', '설렁탕', '옹심이만두국', '콩비지찌개', '열무된장국', '잔치국수', '닭개장',
       '버섯들깨탕', '오징어국', '대구찌개', '가쯔오장국', '짬뽕', '연두부탕', '달래된장찌개', '계란파국',
       '뼈해장국', '양송이스프', '닭곰탕', '수제비국', '알탕', '미소장국', '냉이된장찌개', '유부장국',
       '쇠고기미역국', '매운버섯국', '쇠고기무국', '유부주머니국', '팽이버섯장국', '들깨미역국', '꽃게탕',
       '팽이버섯국', '들깨버섯탕', '북어국', '부대찌개*라면사리', '가쯔오국', '팽이장국', '해물순두부찌개',
       '쌀국수', '시래기국', '열무물국수', '냉모밀국수', '소고기미역국', '호박잎찌개', '김치찌개', '계란국',
       '김치말이국수', '차돌박이된장찌개', '시금치국', '함박스테이크', '부대찌개', '쇠고기샤브국', '물냉면',
       '느타리버섯국', '동태찌개', '돈육김치찌개', '콩가루배추국', '아욱국', '콩나물국', '짬뽕수제비국',
       '콩국수', '스파게티/크림스프', '김치우동', '무채국', '감자국', '작은밥', '콩나물냉국',
       '로제파스타/스프', '얼갈이된장국', '참치김치찌개', '고추장찌개', '꼬지어묵우동', '무채맑은국',
       '조갯살미역국', '근대된장국', '황태국', '어묵우동', '해물된장찌개', '참치주먹밥', '옥수수스프',
       '북어계란국', '바지락수제비', '해물짬뽕', '목살김치찌개', '시금치된장국', 'None', '매운감자국',
       '대구지리', '매생이굴국', '미역국', '봄동된장국', '매운오징어국', '감자찌개', '들깨무채국', '뼈감자탕',
    

In [32]:
train_dinner['반찬1'].unique()

array(['자반고등어구이', '유산슬', '황태양념구이', '멕시칸샐러드', '닭갈비', '군만두', '미니함박',
       '등갈비김치찜', '쇠불고기', '새우또띠아', '명태조림', '김치전', '떡갈비조림', '해물까스&타르타르소스',
       '매운돼지갈비찜', '교자만두', '삼치구이', '꽁치한마리구이', '버섯불고기', '떡볶이', '미트볼조림',
       '돈육굴소스볶음', '수제고로케*케찹', '생선까스*탈탈소스', '개성감자만두', '코다리무조림', '간장찜닭',
       '참치야채전', '해물청경채볶음', '미니햄버거', '임연수찜', '아귀콩나물찜', '찹쌀순대볶음', '동그랑땡전',
       '누룽지탕수육', '돈육씨앗강정', '또띠아피자', '순살깐풍기', '고등어조림', '감자치즈구이', '찜닭',
       '야채튀김', '가자미찜', '수제돈가스', '새싹피자', '닭볶음탕', '해물까스*탈탈소스', '탕수어',
       '쇠고기퀘사디아', '비빔메밀국수', '고추잡채*꽃빵', '고추장불고기', '탕수육', '수제미니햄버거',
       '가자미튀김', '해물누룽지탕', '코다리강정', '주꾸미볶음', '사천식탕수육', '꽁치허브구이', '데리야끼파닭',
       '고구마치즈구이', '춘권*칠리소스', '돈육청경채볶음', '미니함박조림', '수제돈까스', '미트볼케찹조림',
       '오징어무침', '삼치데리야끼구이', '훈제오리냉채', '치킨핑거*요거트D', '비빔국수', '모둠장조림',
       '탕수만두', '감자버터구이', '깐풍기', '오징어볶음', '단호박스프', '수제핫도그', '과일샐러드',
       '쇠고기장조림', '고등어카레구이', '너비아니*부추생채', '치킨핑거', '돈까스', '순대야채볶음',
       '등뼈묵은지찜', '소불고기', '소고기퀘사디아', '쭈꾸미볶음', '치킨텐더', '삼치된장구이', '왕새우튀김',
       '훈제오리/무쌈/양파절임', 

In [33]:
train_dinner['반찬2'].unique()

array(['두부조림', '아삭고추무침', '고기전', '무피클', '감자소세지볶음', '과일샐러드', '어묵볶음',
       '임연수구이', '해파리겨자채', '쨔샤이무침', '감자채볶음', '야채스틱', '청포묵무침', '볶음쌀국수',
       '멸치호두볶음', '멕시칸샐러드', '꽃맛살볶음', '두부양념조림', '콩나물겨자채', '단무지', '옥수수전',
       '야채계란찜', '타코야끼', '고추잡채*꽃빵', '통도라지구이', '꽃맛살샐러드', '버섯메밀전', '단호박범벅',
       '어묵곤약볶음', '새송이버섯볶음', '궁중떡찜', '마파두부', '콘샐러드', '해물까스*탈탈소스', '해초무침',
       '계란찜', '탕평채', '오복지', '계란야채말이', '단무지무침', '견과류조림', '감자범벅', '새송이버섯전',
       '쫄면무침', '모둠묵*양념장', '오이무침', '비엔나케찹볶음', '탕수만두', '두부계란구이', '모듬양채쌈',
       '삼치구이', '오이소박이', '모둠소시지구이', '감자만두', '치즈계란말이', '마늘쫑무침', '잡채',
       '두부계란부침', '군만두', '메추리알조림', '콩나물무침', '락교', '계란말이', '도토리묵무침',
       '감자채햄볶음', '땅콩조림', '단호박범벅/오이피클', '우엉잡채', '야채튀김', '햄맛살볶음', '떡볶이',
       '해물가스*탈탈소스', '연근땅콩조림', '오이피클', '주꾸미초무침', '비엔나컬리플라워볶음', '새우까스*탈탈소스',
       '옥수수감자범벅', '동그랑땡전', '홍시드레싱샐러드', '베이컨계란말이', '무초절임', '비엔나감자볶음',
       '실곤약초무침', '크랜베리단호박샐러드', '꽈리고추어묵조림', '열대과일샐러드', '모둠버섯볶음', '연두부*양념',
       '해물파전', '날치알계란찜', '가자미찜', '단무지/락교', '양장피잡채', '두부구이*양념장', '찐감자',
     

In [34]:
train_dinner['반찬3'].unique()

array(['건파래무침', 'None', '새송이버섯볶음', '콩나물무침', '락교', '물파래무침', '브로컬리초장',
       '봄동무침', '쌈만두', '부추오이생채', '미역레몬초무침', '숙주나물', '무피클', '청경채사과무침',
       '구이김', '양념깻잎지', '부추생채', '오복지', '꼬들단무지', '봄동겉절이', '참나물생채', '오이무침',
       '야채스틱*쌈장', '돌나물초장', '두반장가지나물', '시금치나물', '청경채무침', '도라지나물', '애호박나물',
       '오이피클', '숙주미나리나물', '단무지', '청경채생채', '치커리사과무침', '돌나물오이무침', '피클',
       '취나물땅콩무침', '세발나물', '참나물', '시금치초장무침', '날치알계란찜', '미역줄기볶음', '얼갈이나물',
       '시금치무침', '참나물무침', '아삭고추무침', '쨔샤이무침', '연두부*새싹장', '양배추쌈', '주꾸미무침',
       '마늘쫑무침', '상추겉절이', '오이도라지생채', '스틱단무지', '치커리유자무침', '찐햇감자', '미역줄기',
       '파래김*양념장', '우엉조림', '도토리묵무침', '청경채겉절이', '양배추피클', '김구이', '오이지',
       '단무지무침', '온두부', '가지나물', '메밀비빔국수', '타코야끼', '핫도그', '쑥갓두부무침',
       '양배추쌈*쌈장', '머위대나물', '부추무침', '단배추나물', '수제피클', '무말랭이무침', '잔멸치볶음',
       '도라지오이생채', '쫄면/비트무절임', '해초무침', '노각무침', '호박나물', '셀프무쌈말이', '오이맛살초무침',
       '미역무침/콘샐러드', '유채나물', '매운어묵볶음', '상추치커리무침', '취나물', '상추초무침', '도라지초무침',
       '참나물땅콩무침', '콩자반', '파래김*장', '고구마맛탕', '핫도그&케찹', '비트무생채', '얼갈이겉절이',


In [35]:
train_dinner['반찬4'].unique()

array(['None', '무피클', '단무지', '취나물', '비트무생채', '부추생채', '오이소박이', '쪽파김무침',
       '토마토설탕절인', '연근흑임자샐러드', '브로컬리초회', '다시마채무초무침', '열무김치볶음', '무생채'],
      dtype=object)

In [36]:
train_dinner['디저트1'].unique()

array(['None', '바나나', '군고구마', '요플레', '음료', '깨찰빵', '조각사과', '오렌지', '피크닉',
       '사과', '컵주스', '또띠아칩', '슈크림', '방울토마토', '수박', '토마토', '찐옥수수', '츄러스',
       '참외', '포도', '포도주스', '주스', '요구르트', '푸딩', '호빵', '귤', '아이스슈', '과일',
       '조각티라미수', '애플망고두유', '내사랑포도', '딸기생크림와플', '마늘빵', '오렌지주스', '바나나와플',
       '모둠과일', '매실쥬스', '*천도복숭아', '매실주스', '딸기와플', '청포도', '천도복숭아', '과일주스',
       '사과주스', '파인애플주스', '아오리사과', '사과푸딩', '베리베리퐁당요플레', '망고주스', '배',
       '꽈배기도너츠', '도넛츠'], dtype=object)

In [37]:
train_dinner['디저트2'].unique()  

array(['None', '오렌지쥬스', '오렌지', '참외', '주스', '식혜', '피클', '음료', '오렌지주스',
       '하루야채주스', '요구르트'], dtype=object)

In [38]:
train_dinner['김치'].unique()

array(['포기김치', '깍두기', '석박지', '통배추겉절이', '열무김치', '오이소박이', '배추겉절이', '알타리김치',
       '김치볶음', 'None', '봄동겉절이', '무생채', '배추김치', '갓김치', '겉절이김치', '얼갈이열무겉절이',
       '수제보쌈김치', '백김치', '갓김치/겉절이김치'], dtype=object)

In [39]:
train_dinner['디저트1'].value_counts()

None         971
음료           55 
과일           19 
요플레          17 
바나나          17 
오렌지          17 
귤            10 
수박           9  
요구르트         9  
조각사과         7  
군고구마         6  
주스           5  
방울토마토        5  
피크닉          5  
찐옥수수         4  
오렌지주스        3  
매실주스         3  
사과주스         2  
모둠과일         2  
파인애플주스       2  
호빵           2  
아이스슈         2  
사과           2  
슈크림          2  
또띠아칩         2  
포도주스         1  
컵주스          1  
꽈배기도너츠       1  
배            1  
망고주스         1  
베리베리퐁당요플레    1  
사과푸딩         1  
아오리사과        1  
깨찰빵          1  
과일주스         1  
천도복숭아        1  
청포도          1  
딸기와플         1  
*천도복숭아       1  
푸딩           1  
매실쥬스         1  
바나나와플        1  
마늘빵          1  
딸기생크림와플      1  
내사랑포도        1  
애플망고두유       1  
조각티라미수       1  
토마토          1  
츄러스          1  
참외           1  
포도           1  
도넛츠          1  
Name: 디저트1, dtype: int64

## 분류 후 점심, 저녁 데이터

In [40]:
train_lunch.head()

,일자,중식메뉴,중식계,중식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,반찬5,디저트1,디저트2,김치
0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 청포묵무침 요구르트 포기김치 (배추,고추가루:국내산)",1039.0,"[쌀밥/잡곡밥, 오징어찌개, 쇠불고기, 계란찜, 청포묵무침, 요구르트, 포기김치]",쌀밥/잡곡밥,오징어찌개,쇠불고기,계란찜,청포묵무침,None,None,요구르트,None,포기김치
1,2016-02-02,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉절이 (배추,고추가루:국내산)",867.0,"[쌀밥/잡곡밥, 김치찌개, 가자미튀김, 모둠소세지구이, 마늘쫑무침, 요구르트, 배추겉절이]",쌀밥/잡곡밥,김치찌개,가자미튀김,모둠소세지구이,마늘쫑무침,None,None,요구르트,None,배추겉절이
2,2016-02-03,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 견과류조림 요구르트 포기김치 (배추,고추가루:국내산)",1017.0,"[카레덮밥, 팽이장국, 치킨핑거, 쫄면야채무침, 견과류조림, 요구르트, 포기김치]",카레덮밥,팽이장국,치킨핑거,쫄면야채무침,견과류조림,None,None,요구르트,None,포기김치
3,2016-02-04,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 요구르트 포기김치 (배추,고추가루:국내산)",978.0,"[쌀밥/잡곡밥, 쇠고기무국, 주꾸미볶음, 부추전, 시금치나물, 요구르트, 포기김치]",쌀밥/잡곡밥,쇠고기무국,주꾸미볶음,부추전,시금치나물,None,None,요구르트,None,포기김치
4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채 청경채무침 요구르트 포기김치 (배추,고추가루:국내산)",925.0,"[쌀밥/잡곡밥, 떡국, 돈육씨앗강정, 우엉잡채, 청경채무침, 요구르트, 포기김치]",쌀밥/잡곡밥,떡국,돈육씨앗강정,우엉잡채,청경채무침,None,None,요구르트,None,포기김치


In [41]:
train_dinner.head()

,일자,석식메뉴,석식계,석식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,디저트1,디저트2,김치
0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 포기김치 (김치:국내산)",331.0,"[쌀밥/잡곡밥, 육개장, 자반고등어구이, 두부조림, 건파래무침, 포기김치]",쌀밥/잡곡밥,육개장,자반고등어구이,두부조림,건파래무침,None,None,None,포기김치
1,2016-02-02,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무침 바나나 포기김치 (배추,고추가루:국내산)",560.0,"[콩나물밥*양념장, 어묵국, 유산슬, 아삭고추무침, 바나나, 포기김치]",콩나물밥*양념장,어묵국,유산슬,아삭고추무침,None,None,바나나,None,포기김치
2,2016-02-03,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기전 (돼지고기:국내산) 새송이버섯볶음 포기김치 (배추,고추가루:국내산)",573.0,"[쌀밥/잡곡밥, 청국장찌개, 황태양념구이, 고기전, 새송이버섯볶음, 포기김치]",쌀밥/잡곡밥,청국장찌개,황태양념구이,고기전,새송이버섯볶음,None,None,None,포기김치
3,2016-02-04,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포기김치 (배추,고추가루:국내산)",525.0,"[미니김밥*겨자장, 우동, 멕시칸샐러드, 군고구마, 무피클, 포기김치]",미니김밥*겨자장,우동,멕시칸샐러드,무피클,None,None,군고구마,None,포기김치
4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:국내산) 감자소세지볶음 콩나물무침 포기김치 (배추,고추가루:국내산)",330.0,"[쌀밥/잡곡밥, 차돌박이찌개, 닭갈비, 감자소세지볶음, 콩나물무침, 포기김치]",쌀밥/잡곡밥,차돌박이찌개,닭갈비,감자소세지볶음,콩나물무침,None,None,None,포기김치


In [42]:
#석식 제공하는 날의 데이터
train_dinner_exist = train_dinner[train_dinner['밥']!='No dinner'].reset_index()
train_dinner_exist.head()

,index,일자,석식메뉴,석식계,석식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,디저트1,디저트2,김치
0,0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 포기김치 (김치:국내산)",331.0,"[쌀밥/잡곡밥, 육개장, 자반고등어구이, 두부조림, 건파래무침, 포기김치]",쌀밥/잡곡밥,육개장,자반고등어구이,두부조림,건파래무침,None,None,None,포기김치
1,1,2016-02-02,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무침 바나나 포기김치 (배추,고추가루:국내산)",560.0,"[콩나물밥*양념장, 어묵국, 유산슬, 아삭고추무침, 바나나, 포기김치]",콩나물밥*양념장,어묵국,유산슬,아삭고추무침,None,None,바나나,None,포기김치
2,2,2016-02-03,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기전 (돼지고기:국내산) 새송이버섯볶음 포기김치 (배추,고추가루:국내산)",573.0,"[쌀밥/잡곡밥, 청국장찌개, 황태양념구이, 고기전, 새송이버섯볶음, 포기김치]",쌀밥/잡곡밥,청국장찌개,황태양념구이,고기전,새송이버섯볶음,None,None,None,포기김치
3,3,2016-02-04,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포기김치 (배추,고추가루:국내산)",525.0,"[미니김밥*겨자장, 우동, 멕시칸샐러드, 군고구마, 무피클, 포기김치]",미니김밥*겨자장,우동,멕시칸샐러드,무피클,None,None,군고구마,None,포기김치
4,4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:국내산) 감자소세지볶음 콩나물무침 포기김치 (배추,고추가루:국내산)",330.0,"[쌀밥/잡곡밥, 차돌박이찌개, 닭갈비, 감자소세지볶음, 콩나물무침, 포기김치]",쌀밥/잡곡밥,차돌박이찌개,닭갈비,감자소세지볶음,콩나물무침,None,None,None,포기김치


In [43]:
train_dinner_exist.shape

(1163, 14)

## 테스트 데이터 전처리

In [44]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자전 양상추샐러드/사과 포기김치,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추샐러드*딸기D,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 양상추샐러드 포기김치,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상추샐러드*오미자D,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양상추샐러드 포기김치,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상추샐러드*파인요거트D,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완자전 양상추샐러드 포기김치,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치 양상추샐러드*망고D,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양상추샐러드 포기김치,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸딩 양상추샐러드*오리엔탈D,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [45]:
test_lunch = test[['일자','중식메뉴']]
test_dinner = test[['일자','석식메뉴']]
test_lunch.head()

,일자,중식메뉴
0,2021-01-27,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추샐러드*딸기D
1,2021-01-28,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상추샐러드*오미자D
2,2021-01-29,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상추샐러드*파인요거트D
3,2021-02-01,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치 양상추샐러드*망고D
4,2021-02-02,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸딩 양상추샐러드*오리엔탈D


In [46]:
test_lunch['중식메뉴리스트'] = split_menu(test_lunch['중식메뉴'])
test_dinner['석식메뉴리스트'] = split_menu(test_dinner['석식메뉴'])

/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/1014200251.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_lunch['중식메뉴리스트'] = split_menu(test_lunch['중식메뉴'])
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/1014200251.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dinner['석식메뉴리스트'] = split_menu(test_dinner['석식메뉴'])


In [47]:
test_lunch[['일자','중식메뉴리스트']]

,일자,중식메뉴리스트
0,2021-01-27,"[쌀밥/흑미밥/찰현미밥, 대구지리, 매운돈갈비찜, 오꼬노미계란말이, 상추무침, 포기김치, 양상추샐러드*딸기D]"
1,2021-01-28,"[쌀밥/보리밥/찰현미밥, 우렁된장찌개, 오리주물럭, 청양부추전, 수제삼색무쌈, 겉절이김치, 양상추샐러드*오미자D]"
2,2021-01-29,"[쌀밥/흑미밥/찰현미밥, 팽이장국, 수제돈까스*소스, 가자미조림, 동초나물무침, 포기김치, 양상추샐러드*파인요거트D]"
3,2021-02-01,"[쌀밥/흑미밥/찰현미밥, 배추들깨국, 오리대패불고기, 시금치프리타타, 부추고추장무침, 포기김치, 양상추샐러드*망고D]"
4,2021-02-02,"[쌀밥/팥밥/찰현미밥, 부대찌개, 닭살데리야끼조림, 버섯탕수, 세발나물무침, 알타리김치/사과푸딩, 양상추샐러드*오리엔탈D]"
5,2021-02-03,"[쌀밥/흑미밥/찰현미밥, 아욱국, 매콤해물볶음, 감자조림, 미나리나물, 포기김치, 콥샐러드*렌치D]"
6,2021-02-04,"[쌀밥/차조밥/찰현미밥, 설렁탕, 고등어김치말이찜, 볼어묵굴소스볶음, 브로콜리숙회*초장, 석박지, 양상추샐러드*키위D]"
7,2021-02-05,"[쌀밥/흑미밥/찰현미밥, 북엇국, 닭볶음탕, 채소전*장, 솎음열무나물무침, 포기김치, 양상추샐러드*황도D]"
8,2021-02-08,"[쌀밥/흑미밥/찰현미밥, 감자양파국, 돈수육*씨앗쌈장, 매콤어묵볶음, 콩나물파채무침, 포기김치, 양상추샐러드*자몽D]"
9,2021-02-09,"[쌀밥/기장밥/찰현미밥, 장각백숙, 적어양념장구이, 채소스틱*쌈장, 도라지오이초무침, 겉절이김치, 양상추샐러드*참깨D]"


In [48]:
def seperate_lunch_menu_list_ts(df):

  str = 'None'
  lunch_menu_list = df['중식메뉴리스트']

  print('######## 점심 데이터 구분 함수 시작 ########')

  #메뉴 종류별 구분 (10 피처)
  rice = []
  soup = []
  side_1 = []
  side_2 = []
  side_3 = []
  side_4 = []
  side_5 = []
  dessert_1 = []
  dessert_2 = []
  kimchi = []

  for idx, lunch_menu in enumerate(lunch_menu_list):
    print("현재 인덱스 :", idx)
    # print(lunch_menu)

    #[밥 국 반찬3개 김치/디저트 반찬]
    if idx in [4,12,16,22,30,36,40,43]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[6])
      side_5.append(str)
      dessert_1.append(lunch_menu[5].split('/')[1])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5].split('/')[0])
    #[밥 국 반찬2개 김치/디저트 반찬]
    elif idx in [20]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[5])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4].split('/')[1])
      dessert_2.append(str)
      kimchi.append(lunch_menu[4].split('/')[0])
    #[밥 국 반찬2개 김치 반찬]
    elif idx in [25,34,48]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[5])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[4])
    #[밥 국 반찬2개 디저트 김치 반찬]
    elif idx in [27]:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[6])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(lunch_menu[4])
      dessert_2.append(str)
      kimchi.append(lunch_menu[5])
    #[밥 국 반찬3개 김치 반찬]
    else:   
      rice.append(lunch_menu[0])
      soup.append(lunch_menu[1])
      side_1.append(lunch_menu[2])
      side_2.append(lunch_menu[3])
      side_3.append(lunch_menu[4])
      side_4.append(lunch_menu[6])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(lunch_menu[5])

  print('######## 점심 데이터 구분 함수 종료 ########')

  return rice, soup, side_1, side_2, side_3, side_4, side_5, dessert_1, dessert_2, kimchi
  

In [49]:
rice, soup, side_1, side_2, side_3, side_4, side_5, dessert_1, dessert_2, kimchi = seperate_lunch_menu_list_ts(test_lunch)

######## 점심 데이터 구분 함수 시작 ########
현재 인덱스 : 0
현재 인덱스 : 1
현재 인덱스 : 2
현재 인덱스 : 3
현재 인덱스 : 4
현재 인덱스 : 5
현재 인덱스 : 6
현재 인덱스 : 7
현재 인덱스 : 8
현재 인덱스 : 9
현재 인덱스 : 10
현재 인덱스 : 11
현재 인덱스 : 12
현재 인덱스 : 13
현재 인덱스 : 14
현재 인덱스 : 15
현재 인덱스 : 16
현재 인덱스 : 17
현재 인덱스 : 18
현재 인덱스 : 19
현재 인덱스 : 20
현재 인덱스 : 21
현재 인덱스 : 22
현재 인덱스 : 23
현재 인덱스 : 24
현재 인덱스 : 25
현재 인덱스 : 26
현재 인덱스 : 27
현재 인덱스 : 28
현재 인덱스 : 29
현재 인덱스 : 30
현재 인덱스 : 31
현재 인덱스 : 32
현재 인덱스 : 33
현재 인덱스 : 34
현재 인덱스 : 35
현재 인덱스 : 36
현재 인덱스 : 37
현재 인덱스 : 38
현재 인덱스 : 39
현재 인덱스 : 40
현재 인덱스 : 41
현재 인덱스 : 42
현재 인덱스 : 43
현재 인덱스 : 44
현재 인덱스 : 45
현재 인덱스 : 46
현재 인덱스 : 47
현재 인덱스 : 48
현재 인덱스 : 49
######## 점심 데이터 구분 함수 종료 ########


In [50]:
test_lunch['밥'] = rice
test_lunch['국'] = soup
test_lunch['반찬1'] = side_1
test_lunch['반찬2'] = side_2
test_lunch['반찬3'] = side_3
test_lunch['반찬4'] = side_4
test_lunch['반찬5'] = side_5
test_lunch['디저트1'] = dessert_1
test_lunch['디저트2'] = dessert_2
test_lunch['김치'] = kimchi

/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/4040943376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_lunch['밥'] = rice
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/4040943376.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_lunch['국'] = soup
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/4040943376.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [51]:
test_lunch

,일자,중식메뉴,중식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,반찬5,디저트1,디저트2,김치
0,2021-01-27,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추샐러드*딸기D,"[쌀밥/흑미밥/찰현미밥, 대구지리, 매운돈갈비찜, 오꼬노미계란말이, 상추무침, 포기김치, 양상추샐러드*딸기D]",쌀밥/흑미밥/찰현미밥,대구지리,매운돈갈비찜,오꼬노미계란말이,상추무침,양상추샐러드*딸기D,None,None,None,포기김치
1,2021-01-28,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상추샐러드*오미자D,"[쌀밥/보리밥/찰현미밥, 우렁된장찌개, 오리주물럭, 청양부추전, 수제삼색무쌈, 겉절이김치, 양상추샐러드*오미자D]",쌀밥/보리밥/찰현미밥,우렁된장찌개,오리주물럭,청양부추전,수제삼색무쌈,양상추샐러드*오미자D,None,None,None,겉절이김치
2,2021-01-29,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상추샐러드*파인요거트D,"[쌀밥/흑미밥/찰현미밥, 팽이장국, 수제돈까스*소스, 가자미조림, 동초나물무침, 포기김치, 양상추샐러드*파인요거트D]",쌀밥/흑미밥/찰현미밥,팽이장국,수제돈까스*소스,가자미조림,동초나물무침,양상추샐러드*파인요거트D,None,None,None,포기김치
3,2021-02-01,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치 양상추샐러드*망고D,"[쌀밥/흑미밥/찰현미밥, 배추들깨국, 오리대패불고기, 시금치프리타타, 부추고추장무침, 포기김치, 양상추샐러드*망고D]",쌀밥/흑미밥/찰현미밥,배추들깨국,오리대패불고기,시금치프리타타,부추고추장무침,양상추샐러드*망고D,None,None,None,포기김치
4,2021-02-02,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸딩 양상추샐러드*오리엔탈D,"[쌀밥/팥밥/찰현미밥, 부대찌개, 닭살데리야끼조림, 버섯탕수, 세발나물무침, 알타리김치/사과푸딩, 양상추샐러드*오리엔탈D]",쌀밥/팥밥/찰현미밥,부대찌개,닭살데리야끼조림,버섯탕수,세발나물무침,양상추샐러드*오리엔탈D,None,사과푸딩,None,알타리김치
5,2021-02-03,쌀밥/흑미밥/찰현미밥 아욱국 매콤해물볶음 감자조림 미나리나물 포기김치 콥샐러드*렌치D,"[쌀밥/흑미밥/찰현미밥, 아욱국, 매콤해물볶음, 감자조림, 미나리나물, 포기김치, 콥샐러드*렌치D]",쌀밥/흑미밥/찰현미밥,아욱국,매콤해물볶음,감자조림,미나리나물,콥샐러드*렌치D,None,None,None,포기김치
6,2021-02-04,쌀밥/차조밥/찰현미밥 설렁탕 고등어김치말이찜 볼어묵굴소스볶음 브로콜리숙회*초장 석박지 양상추샐러드*키위D,"[쌀밥/차조밥/찰현미밥, 설렁탕, 고등어김치말이찜, 볼어묵굴소스볶음, 브로콜리숙회*초장, 석박지, 양상추샐러드*키위D]",쌀밥/차조밥/찰현미밥,설렁탕,고등어김치말이찜,볼어묵굴소스볶음,브로콜리숙회*초장,양상추샐러드*키위D,None,None,None,석박지
7,2021-02-05,쌀밥/흑미밥/찰현미밥 북엇국 닭볶음탕 채소전*장 솎음열무나물무침 포기김치 양상추샐러드*황도D,"[쌀밥/흑미밥/찰현미밥, 북엇국, 닭볶음탕, 채소전*장, 솎음열무나물무침, 포기김치, 양상추샐러드*황도D]",쌀밥/흑미밥/찰현미밥,북엇국,닭볶음탕,채소전*장,솎음열무나물무침,양상추샐러드*황도D,None,None,None,포기김치
8,2021-02-08,쌀밥/흑미밥/찰현미밥 감자양파국 돈수육*씨앗쌈장 매콤어묵볶음 콩나물파채무침 포기김치 양상추샐러드*자몽D,"[쌀밥/흑미밥/찰현미밥, 감자양파국, 돈수육*씨앗쌈장, 매콤어묵볶음, 콩나물파채무침, 포기김치, 양상추샐러드*자몽D]",쌀밥/흑미밥/찰현미밥,감자양파국,돈수육*씨앗쌈장,매콤어묵볶음,콩나물파채무침,양상추샐러드*자몽D,None,None,None,포기김치
9,2021-02-09,쌀밥/기장밥/찰현미밥 장각백숙 적어양념장구이 채소스틱*쌈장 도라지오이초무침 겉절이김치 양상추샐러드*참깨D,"[쌀밥/기장밥/찰현미밥, 장각백숙, 적어양념장구이, 채소스틱*쌈장, 도라지오이초무침, 겉절이김치, 양상추샐러드*참깨D]",쌀밥/기장밥/찰현미밥,장각백숙,적어양념장구이,채소스틱*쌈장,도라지오이초무침,양상추샐러드*참깨D,None,None,None,겉절이김치


In [52]:
test_dinner[['일자','석식메뉴리스트']]

,일자,석식메뉴리스트
0,2021-01-27,"[흑미밥, 얼큰순두부찌개, 쇠고기우엉볶음, 버섯햄볶음, 포기김치]"
1,2021-01-28,"[충무김밥, 우동국물, 오징어무침, 꽃맛살샐러드, 얼갈이쌈장무침, 석박지]"
2,2021-01-29,"[흑미밥, 물만둣국, 카레찜닭, 숯불양념꼬지어묵, 꼬시래기무침, 포기김치]"
3,2021-02-01,"[흑미밥, 동태탕, 돈육꽈리고추장조림, 당면채소무침, 모자반무침, 포기김치]"
4,2021-02-02,"[흑미밥, 바지락살국, 쇠고기청경채볶음, 두부구이*볶은김치, 머위된장무침, 백김치]"
5,2021-02-03,"[오므라이스, 가쓰오장국, 빌소세지구이*구운채소, 단감치커리무침, 양념고추지, 겉절이김치]"
6,2021-02-04,"[흑미밥, 계란파국, 돈육두루치기, 감자채파프리카볶음, 세발나물오리엔탈무침, 포기김치]"
7,2021-02-05,"[유부초밥/추가밥, 온메밀소바, 국물떡볶이, 순대찜*소금, 청경채겉절이, 포기김치]"
8,2021-02-08,"[흑미밥, 냉이국, 반반치킨, 꼬막채소무침, 청경채찜, 포기김치]"
9,2021-02-09,"[흑미밥, 미역국, 매운소불고기, 단호박두부탕수, 메추리알장조림, 석박지]"


In [53]:
def seperate_dinner_menu_list_ts(df):

  str = 'None'
  dinner_menu_list = df['석식메뉴리스트']

  print('######## 저녁 데이터 구분 함수 시작 ########')

  #메뉴 종류별 구분 (10 피처)
  rice = []
  soup = []
  side_1 = []
  side_2 = []
  side_3 = []
  side_4 = []
  side_5 = []
  dessert_1 = []
  dessert_2 = []
  kimchi = []

  for idx, dinner_menu in enumerate(dinner_menu_list):
    print("현재 인덱스 :", idx)
    print(dinner_menu)

    #[밥 국 반찬2개 김치]
    if idx in [0,24,33]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[4])
    #[밥 국 반찬2개 디저트 김치]
    elif idx in [47]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(str)
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(dinner_menu[4])
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])
    #[밥 반찬3개 김치]
    elif idx in [22]:   
      rice.append(dinner_menu[0])
      soup.append(str)
      side_1.append(dinner_menu[1])
      side_2.append(dinner_menu[2])
      side_3.append(dinner_menu[3])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[4])
    #[밥 국 반찬4개 김치]
    elif idx in [32]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4])
      side_4.append(dinner_menu[5])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[6])
    #[]
    elif idx in [37]:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[5])
      side_4.append(dinner_menu[6])
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[7])
    #[밥 국 반찬3개 김치]
    else:   
      rice.append(dinner_menu[0])
      soup.append(dinner_menu[1])
      side_1.append(dinner_menu[2])
      side_2.append(dinner_menu[3])
      side_3.append(dinner_menu[4])
      side_4.append(str)
      side_5.append(str)
      dessert_1.append(str)
      dessert_2.append(str)
      kimchi.append(dinner_menu[5])
  
  print('######## 저녁 데이터 구분 함수 종료 ########')

  return rice, soup, side_1, side_2, side_3, side_4, dessert_1, dessert_2, kimchi
  

In [54]:
rice, soup, side_1, side_2, side_3, side_4, dessert_1, dessert_2, kimchi = seperate_dinner_menu_list_ts(test_dinner)

######## 저녁 데이터 구분 함수 시작 ########
현재 인덱스 : 0
['흑미밥', '얼큰순두부찌개', '쇠고기우엉볶음', '버섯햄볶음', '포기김치']
현재 인덱스 : 1
['충무김밥', '우동국물', '오징어무침', '꽃맛살샐러드', '얼갈이쌈장무침', '석박지']
현재 인덱스 : 2
['흑미밥', '물만둣국', '카레찜닭', '숯불양념꼬지어묵', '꼬시래기무침', '포기김치']
현재 인덱스 : 3
['흑미밥', '동태탕', '돈육꽈리고추장조림', '당면채소무침', '모자반무침', '포기김치']
현재 인덱스 : 4
['흑미밥', '바지락살국', '쇠고기청경채볶음', '두부구이*볶은김치', '머위된장무침', '백김치']
현재 인덱스 : 5
['오므라이스', '가쓰오장국', '빌소세지구이*구운채소', '단감치커리무침', '양념고추지', '겉절이김치']
현재 인덱스 : 6
['흑미밥', '계란파국', '돈육두루치기', '감자채파프리카볶음', '세발나물오리엔탈무침', '포기김치']
현재 인덱스 : 7
['유부초밥/추가밥', '온메밀소바', '국물떡볶이', '순대찜*소금', '청경채겉절이', '포기김치']
현재 인덱스 : 8
['흑미밥', '냉이국', '반반치킨', '꼬막채소무침', '청경채찜', '포기김치']
현재 인덱스 : 9
['흑미밥', '미역국', '매운소불고기', '단호박두부탕수', '메추리알장조림', '석박지']
현재 인덱스 : 10
['흑미밥', '참치김치찌개', '오징어굴소스볶음', '차돌비빔국수', '건새우무나물', '포기김치']
현재 인덱스 : 11
['흑미밥', '순두부백탕', '수제치킨까스', '쫄면채소무침', '얼갈이나물', '포기김치']
현재 인덱스 : 12
['흑미밥', '손수제비국', '쇠고기낙지볶음', '카레홍합찜', '쑥갓나물', '포기김치']
현재 인덱스 : 13
['곤드레밥', '황태국', '찰떡떡갈비조림', '계란후라이', '재래김*달래양념장', '무생채']
현재 인덱스 : 14
['흑미밥', '바지락된장찌개', '

In [55]:
test_dinner['밥'] = rice
test_dinner['국'] = soup
test_dinner['반찬1'] = side_1
test_dinner['반찬2'] = side_2
test_dinner['반찬3'] = side_3
test_dinner['반찬4'] = side_4
test_dinner['디저트1'] = dessert_1
test_dinner['디저트2'] = dessert_2
test_dinner['김치'] = kimchi

/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/1828890395.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dinner['밥'] = rice
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/1828890395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dinner['국'] = soup
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/1828890395.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [56]:
test_dinner

,일자,석식메뉴,석식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,디저트1,디저트2,김치
0,2021-01-27,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,"[흑미밥, 얼큰순두부찌개, 쇠고기우엉볶음, 버섯햄볶음, 포기김치]",흑미밥,얼큰순두부찌개,쇠고기우엉볶음,버섯햄볶음,None,None,None,None,포기김치
1,2021-01-28,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,"[충무김밥, 우동국물, 오징어무침, 꽃맛살샐러드, 얼갈이쌈장무침, 석박지]",충무김밥,우동국물,오징어무침,꽃맛살샐러드,얼갈이쌈장무침,None,None,None,석박지
2,2021-01-29,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치,"[흑미밥, 물만둣국, 카레찜닭, 숯불양념꼬지어묵, 꼬시래기무침, 포기김치]",흑미밥,물만둣국,카레찜닭,숯불양념꼬지어묵,꼬시래기무침,None,None,None,포기김치
3,2021-02-01,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치,"[흑미밥, 동태탕, 돈육꽈리고추장조림, 당면채소무침, 모자반무침, 포기김치]",흑미밥,동태탕,돈육꽈리고추장조림,당면채소무침,모자반무침,None,None,None,포기김치
4,2021-02-02,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치,"[흑미밥, 바지락살국, 쇠고기청경채볶음, 두부구이*볶은김치, 머위된장무침, 백김치]",흑미밥,바지락살국,쇠고기청경채볶음,두부구이*볶은김치,머위된장무침,None,None,None,백김치
5,2021-02-03,오므라이스 가쓰오장국 빌소세지구이*구운채소 단감치커리무침 양념고추지 겉절이김치,"[오므라이스, 가쓰오장국, 빌소세지구이*구운채소, 단감치커리무침, 양념고추지, 겉절이김치]",오므라이스,가쓰오장국,빌소세지구이*구운채소,단감치커리무침,양념고추지,None,None,None,겉절이김치
6,2021-02-04,흑미밥 계란파국 돈육두루치기 감자채파프리카볶음 세발나물오리엔탈무침 포기김치,"[흑미밥, 계란파국, 돈육두루치기, 감자채파프리카볶음, 세발나물오리엔탈무침, 포기김치]",흑미밥,계란파국,돈육두루치기,감자채파프리카볶음,세발나물오리엔탈무침,None,None,None,포기김치
7,2021-02-05,유부초밥/추가밥 온메밀소바 국물떡볶이 순대찜*소금 청경채겉절이 포기김치,"[유부초밥/추가밥, 온메밀소바, 국물떡볶이, 순대찜*소금, 청경채겉절이, 포기김치]",유부초밥/추가밥,온메밀소바,국물떡볶이,순대찜*소금,청경채겉절이,None,None,None,포기김치
8,2021-02-08,흑미밥 냉이국 반반치킨 꼬막채소무침 청경채찜 포기김치,"[흑미밥, 냉이국, 반반치킨, 꼬막채소무침, 청경채찜, 포기김치]",흑미밥,냉이국,반반치킨,꼬막채소무침,청경채찜,None,None,None,포기김치
9,2021-02-09,흑미밥 미역국 매운소불고기 단호박두부탕수 메추리알장조림 석박지,"[흑미밥, 미역국, 매운소불고기, 단호박두부탕수, 메추리알장조림, 석박지]",흑미밥,미역국,매운소불고기,단호박두부탕수,메추리알장조림,None,None,None,석박지


## 추가 전처리

### 원핫 인코딩

In [57]:
pd.get_dummies(train_lunch['밥']).shape

(1205, 73)

In [58]:
pd.get_dummies(train_lunch['국']).shape

(1205, 263)

In [59]:
pd.get_dummies(train_lunch['반찬1']).shape

(1205, 386)

In [60]:
pd.get_dummies(train_lunch['반찬2']).shape

(1205, 463)

In [61]:
pd.get_dummies(train_lunch['반찬3']).shape

(1205, 420)

In [62]:
pd.get_dummies(train_lunch['반찬4']).shape

(1205, 90)

In [63]:
pd.get_dummies(train_lunch['반찬5']).shape

(1205, 10)

In [64]:
pd.get_dummies(train_lunch['디저트1']).shape

(1205, 71)

In [65]:
pd.get_dummies(train_lunch['디저트2']).shape

(1205, 12)

In [66]:
pd.get_dummies(train_lunch['김치']).shape

(1205, 28)

데이터 결합하려고 하면 런타임 끊김...

### 중식계가 높은 데이터에 가중치 부여

### 점심

In [67]:
train_lunch['중식계'].describe()

count    1205.000000
mean     890.334440 
std      209.505057 
min      296.000000 
25%      758.000000 
50%      879.000000 
75%      1032.000000
max      1459.000000
Name: 중식계, dtype: float64

In [68]:
list(set(train_lunch[train_lunch['중식계']>1032]['밥'].tolist()))

['쌀밥/오곡밥',
 '마파두부덮밥',
 '쌀밥/기장밥/찰현미밥',
 '비빔밥',
 '나물비빔밥',
 '카레라이스',
 '열무비빔밥',
 '쌀밥/수수밥/찰현미밥',
 '쌀밥/팥밥/찰현미밥',
 '김치제육덮밥',
 'None',
 '쌀밥/차조밥/찰현미밥',
 '콩나물밥',
 '쌀밥/흑미밥/찰현미밥',
 '낙지비빔밥',
 '곤드레밥*강된장',
 '쌀밥/잡곡밥',
 '콩나물밥*양념장',
 '카레덮밥',
 '봄새싹비빔밥',
 '쌀밥/찰보리밥/찰현미밥',
 '짜장밥',
 '찹쌀밥',
 '쌀밥/강낭콩밥/찰현미밥',
 '쌀밥/보리밥/찰현미밥']

In [69]:
rice_75 = list(set(train_lunch[train_lunch['중식계']>1032]['밥'].tolist()))
soup_75 = list(set(train_lunch[train_lunch['중식계']>1032]['국'].tolist()))
side_1_75 = list(set(train_lunch[train_lunch['중식계']>1032]['반찬1'].tolist()))
side_2_75 = list(set(train_lunch[train_lunch['중식계']>1032]['반찬2'].tolist()))
side_3_75 = list(set(train_lunch[train_lunch['중식계']>1032]['반찬3'].tolist()))
side_4_75 = list(set(train_lunch[train_lunch['중식계']>1032]['반찬4'].tolist()))
side_5_75 = list(set(train_lunch[train_lunch['중식계']>1032]['반찬5'].tolist()))
dessert_1_75 = list(set(train_lunch[train_lunch['중식계']>1032]['디저트1'].tolist()))
dessert_2_75 = list(set(train_lunch[train_lunch['중식계']>1032]['디저트2'].tolist()))
kimchi_75 = list(set(train_lunch[train_lunch['중식계']>1032]['김치'].tolist()))

In [70]:
kimchi_75

['백김치',
 '알타리김치',
 '오이소박이',
 '열무김치',
 '석박지',
 '봄동겉절이',
 '배추겉절이',
 '깍두기',
 '포기김치',
 '단호박물김치',
 '겉절이김치']

In [71]:
train_lunch_copy = train_lunch.copy()
train_lunch_copy.head()

,일자,중식메뉴,중식계,중식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,반찬5,디저트1,디저트2,김치
0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 청포묵무침 요구르트 포기김치 (배추,고추가루:국내산)",1039.0,"[쌀밥/잡곡밥, 오징어찌개, 쇠불고기, 계란찜, 청포묵무침, 요구르트, 포기김치]",쌀밥/잡곡밥,오징어찌개,쇠불고기,계란찜,청포묵무침,None,None,요구르트,None,포기김치
1,2016-02-02,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉절이 (배추,고추가루:국내산)",867.0,"[쌀밥/잡곡밥, 김치찌개, 가자미튀김, 모둠소세지구이, 마늘쫑무침, 요구르트, 배추겉절이]",쌀밥/잡곡밥,김치찌개,가자미튀김,모둠소세지구이,마늘쫑무침,None,None,요구르트,None,배추겉절이
2,2016-02-03,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 견과류조림 요구르트 포기김치 (배추,고추가루:국내산)",1017.0,"[카레덮밥, 팽이장국, 치킨핑거, 쫄면야채무침, 견과류조림, 요구르트, 포기김치]",카레덮밥,팽이장국,치킨핑거,쫄면야채무침,견과류조림,None,None,요구르트,None,포기김치
3,2016-02-04,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 요구르트 포기김치 (배추,고추가루:국내산)",978.0,"[쌀밥/잡곡밥, 쇠고기무국, 주꾸미볶음, 부추전, 시금치나물, 요구르트, 포기김치]",쌀밥/잡곡밥,쇠고기무국,주꾸미볶음,부추전,시금치나물,None,None,요구르트,None,포기김치
4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채 청경채무침 요구르트 포기김치 (배추,고추가루:국내산)",925.0,"[쌀밥/잡곡밥, 떡국, 돈육씨앗강정, 우엉잡채, 청경채무침, 요구르트, 포기김치]",쌀밥/잡곡밥,떡국,돈육씨앗강정,우엉잡채,청경채무침,None,None,요구르트,None,포기김치


In [72]:
#train_lunch_copy
for idx, data in enumerate(train_lunch_copy['밥']):
  if data in rice_75:
    train_lunch_copy['밥'][idx]=1
  else:
    train_lunch_copy['밥'][idx]=0

for idx, data in enumerate(train_lunch_copy['국']):
  if data in soup_75:
    train_lunch_copy['국'][idx]=1
  else:
    train_lunch_copy['국'][idx]=0

for idx, data in enumerate(train_lunch_copy['반찬1']):
  if data in side_1_75:
    train_lunch_copy['반찬1'][idx]=1
  else:
    train_lunch_copy['반찬1'][idx]=0

for idx, data in enumerate(train_lunch_copy['반찬2']):
  if data in side_2_75:
    train_lunch_copy['반찬2'][idx]=1
  else:
    train_lunch_copy['반찬2'][idx]=0

for idx, data in enumerate(train_lunch_copy['반찬3']):
  if data in side_3_75:
    train_lunch_copy['반찬3'][idx]=1
  else:
    train_lunch_copy['반찬3'][idx]=0    

for idx, data in enumerate(train_lunch_copy['반찬4']):
  if data in side_4_75:
    train_lunch_copy['반찬4'][idx]=1
  else:
    train_lunch_copy['반찬4'][idx]=0

for idx, data in enumerate(train_lunch_copy['반찬5']):
  if data in side_5_75:
    train_lunch_copy['반찬5'][idx]=1
  else:
    train_lunch_copy['반찬5'][idx]=0

for idx, data in enumerate(train_lunch_copy['디저트1']):
  if data in dessert_1_75:
    train_lunch_copy['디저트1'][idx]=1
  else:
    train_lunch_copy['디저트1'][idx]=0

for idx, data in enumerate(train_lunch_copy['디저트2']):
  if data in dessert_2_75:
    train_lunch_copy['디저트2'][idx]=1
  else:
    train_lunch_copy['디저트2'][idx]=0

for idx, data in enumerate(train_lunch_copy['김치']):
  if data in kimchi_75:
    train_lunch_copy['김치'][idx]=1
  else:
    train_lunch_copy['김치'][idx]=0


/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/3714953641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_lunch_copy['밥'][idx]=1
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/3714953641.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_lunch_copy['밥'][idx]=0
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/3714953641.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [73]:
train_lunch_copy.head()

,일자,중식메뉴,중식계,중식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,반찬5,디저트1,디저트2,김치
0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 청포묵무침 요구르트 포기김치 (배추,고추가루:국내산)",1039.0,"[쌀밥/잡곡밥, 오징어찌개, 쇠불고기, 계란찜, 청포묵무침, 요구르트, 포기김치]",1,1,1,1,1,1,1,1,1,1
1,2016-02-02,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉절이 (배추,고추가루:국내산)",867.0,"[쌀밥/잡곡밥, 김치찌개, 가자미튀김, 모둠소세지구이, 마늘쫑무침, 요구르트, 배추겉절이]",1,0,0,1,1,1,1,1,1,1
2,2016-02-03,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 견과류조림 요구르트 포기김치 (배추,고추가루:국내산)",1017.0,"[카레덮밥, 팽이장국, 치킨핑거, 쫄면야채무침, 견과류조림, 요구르트, 포기김치]",1,1,1,1,1,1,1,1,1,1
3,2016-02-04,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 요구르트 포기김치 (배추,고추가루:국내산)",978.0,"[쌀밥/잡곡밥, 쇠고기무국, 주꾸미볶음, 부추전, 시금치나물, 요구르트, 포기김치]",1,1,1,0,1,1,1,1,1,1
4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채 청경채무침 요구르트 포기김치 (배추,고추가루:국내산)",925.0,"[쌀밥/잡곡밥, 떡국, 돈육씨앗강정, 우엉잡채, 청경채무침, 요구르트, 포기김치]",1,1,0,1,0,1,1,1,1,1


In [74]:
#test_lunch_copy
test_lunch_copy = test_lunch.copy()
for idx, data in enumerate(test_lunch_copy['밥']):
  if data in rice_75:
    test_lunch_copy['밥'][idx]=1
  else:
    test_lunch_copy['밥'][idx]=0

for idx, data in enumerate(test_lunch_copy['국']):
  if data in soup_75:
    test_lunch_copy['국'][idx]=1
  else:
    test_lunch_copy['국'][idx]=0

for idx, data in enumerate(test_lunch_copy['반찬1']):
  if data in side_1_75:
    test_lunch_copy['반찬1'][idx]=1
  else:
    test_lunch_copy['반찬1'][idx]=0

for idx, data in enumerate(test_lunch_copy['반찬2']):
  if data in side_2_75:
    test_lunch_copy['반찬2'][idx]=1
  else:
    test_lunch_copy['반찬2'][idx]=0

for idx, data in enumerate(test_lunch_copy['반찬3']):
  if data in side_3_75:
    test_lunch_copy['반찬3'][idx]=1
  else:
    test_lunch_copy['반찬3'][idx]=0    

for idx, data in enumerate(test_lunch_copy['반찬4']):
  if data in side_4_75:
    test_lunch_copy['반찬4'][idx]=1
  else:
    test_lunch_copy['반찬4'][idx]=0

for idx, data in enumerate(test_lunch_copy['반찬5']):
  if data in side_5_75:
    test_lunch_copy['반찬5'][idx]=1
  else:
    test_lunch_copy['반찬5'][idx]=0

for idx, data in enumerate(test_lunch_copy['디저트1']):
  if data in dessert_1_75:
    test_lunch_copy['디저트1'][idx]=1
  else:
    test_lunch_copy['디저트1'][idx]=0

for idx, data in enumerate(test_lunch_copy['디저트2']):
  if data in dessert_2_75:
    test_lunch_copy['디저트2'][idx]=1
  else:
    test_lunch_copy['디저트2'][idx]=0

for idx, data in enumerate(test_lunch_copy['김치']):
  if data in kimchi_75:
    test_lunch_copy['김치'][idx]=1
  else:
    test_lunch_copy['김치'][idx]=0


In [75]:
test_lunch_copy.head()

,일자,중식메뉴,중식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,반찬5,디저트1,디저트2,김치
0,2021-01-27,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추샐러드*딸기D,"[쌀밥/흑미밥/찰현미밥, 대구지리, 매운돈갈비찜, 오꼬노미계란말이, 상추무침, 포기김치, 양상추샐러드*딸기D]",1,0,0,1,1,1,1,1,1,1
1,2021-01-28,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상추샐러드*오미자D,"[쌀밥/보리밥/찰현미밥, 우렁된장찌개, 오리주물럭, 청양부추전, 수제삼색무쌈, 겉절이김치, 양상추샐러드*오미자D]",1,0,1,0,0,1,1,1,1,1
2,2021-01-29,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상추샐러드*파인요거트D,"[쌀밥/흑미밥/찰현미밥, 팽이장국, 수제돈까스*소스, 가자미조림, 동초나물무침, 포기김치, 양상추샐러드*파인요거트D]",1,1,0,0,0,0,1,1,1,1
3,2021-02-01,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치 양상추샐러드*망고D,"[쌀밥/흑미밥/찰현미밥, 배추들깨국, 오리대패불고기, 시금치프리타타, 부추고추장무침, 포기김치, 양상추샐러드*망고D]",1,0,1,1,0,1,1,1,1,1
4,2021-02-02,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸딩 양상추샐러드*오리엔탈D,"[쌀밥/팥밥/찰현미밥, 부대찌개, 닭살데리야끼조림, 버섯탕수, 세발나물무침, 알타리김치/사과푸딩, 양상추샐러드*오리엔탈D]",1,1,0,0,1,1,1,0,1,1


### 저녁

In [76]:
train_dinner_exist['석식계'].describe()

count    1163.000000
mean     478.063629 
std      110.821783 
min      0.000000   
25%      414.000000 
50%      486.000000 
75%      548.000000 
max      905.000000 
Name: 석식계, dtype: float64

In [77]:
list(set(train_dinner_exist[train_dinner_exist['석식계']>548]['밥'].tolist()))

['통새우김밥',
 '비빔밥',
 '쇠고기덮밥',
 '옛날왕돈까스',
 '꼬마김밥*겨자장',
 '야채볶음밥',
 '볶음밥*짜장소스',
 '나가사끼짬뽕',
 '김치주먹밥',
 '하와이안필라프',
 '볶음밥*자장소스',
 '생선초밥',
 '짜장덮밥',
 '유부초밥',
 '참치주먹밥',
 '날치알김치볶음밥',
 '산채비빔밥',
 '해물굴소스볶음밥',
 '낙지볶음덮밥',
 '후난식볶음밥',
 '흑미밥',
 '버섯영양밥*우렁양념장',
 '닭갈비볶음밥',
 '콩나물밥',
 '토마토스파게티',
 '굴소스파인볶음밥',
 '마늘베이컨볶음밥',
 '오므라이스',
 '김치우동',
 '짜장잡채덮밥',
 '참치마요덮밥',
 '볶음밥',
 '목살필라프',
 '충무김밥',
 '미니햄버거',
 '볶음밥&짜장소스',
 '멸치주먹밥',
 '김치볶음밥/계란',
 '쌀밥/잡곡밥',
 '김밥볶음밥',
 '빠에야',
 '콩나물밥*양념장',
 '쇠고기숙주규동덮밥',
 '짜장면/작은밥',
 '부채살오므라이스',
 '투움바스파게티',
 '카레덮밥',
 '김밥',
 '고추잡채덮밥',
 '마파두부',
 '유산슬덮밥',
 '김치볶음밥',
 '참치덮밥',
 '참치회덮밥',
 '베이컨김치볶음밥',
 '볶음밥/짜장소스',
 '작은밥']

In [78]:
rice_75 = list(set(train_dinner_exist[train_dinner_exist['석식계']>545]['밥'].tolist()))
soup_75 = list(set(train_dinner_exist[train_dinner_exist['석식계']>545]['국'].tolist()))
side_1_75 = list(set(train_dinner_exist[train_dinner_exist['석식계']>545]['반찬1'].tolist()))
side_2_75 = list(set(train_dinner_exist[train_dinner_exist['석식계']>545]['반찬2'].tolist()))
side_3_75 = list(set(train_dinner_exist[train_dinner_exist['석식계']>545]['반찬3'].tolist()))
side_4_75 = list(set(train_dinner_exist[train_dinner_exist['석식계']>545]['반찬4'].tolist()))
dessert_1_75 = list(set(train_dinner_exist[train_dinner_exist['석식계']>545]['디저트1'].tolist()))
dessert_2_75 = list(set(train_dinner_exist[train_dinner_exist['석식계']>545]['디저트2'].tolist()))
kimchi_75 = list(set(train_dinner_exist[train_dinner_exist['석식계']>545]['김치'].tolist()))

In [79]:
#train_dinner_copy
train_dinner_copy = train_dinner_exist.copy()
for idx, data in enumerate(train_dinner_copy['밥']):
  if data in rice_75:
    train_dinner_copy['밥'][idx]=1
  else:
    train_dinner_copy['밥'][idx]=0

for idx, data in enumerate(train_dinner_copy['국']):
  if data in soup_75:
    train_dinner_copy['국'][idx]=1
  else:
    train_dinner_copy['국'][idx]=0

for idx, data in enumerate(train_dinner_copy['반찬1']):
  if data in side_1_75:
    train_dinner_copy['반찬1'][idx]=1
  else:
    train_dinner_copy['반찬1'][idx]=0

for idx, data in enumerate(train_dinner_copy['반찬2']):
  if data in side_2_75:
    train_dinner_copy['반찬2'][idx]=1
  else:
    train_dinner_copy['반찬2'][idx]=0

for idx, data in enumerate(train_dinner_copy['반찬3']):
  if data in side_3_75:
    train_dinner_copy['반찬3'][idx]=1
  else:
    train_dinner_copy['반찬3'][idx]=0    

for idx, data in enumerate(train_dinner_copy['반찬4']):
  if data in side_4_75:
    train_dinner_copy['반찬4'][idx]=1
  else:
    train_dinner_copy['반찬4'][idx]=0

for idx, data in enumerate(train_dinner_copy['디저트1']):
  if data in dessert_1_75:
    train_dinner_copy['디저트1'][idx]=1
  else:
    train_dinner_copy['디저트1'][idx]=0

for idx, data in enumerate(train_dinner_copy['디저트2']):
  if data in dessert_2_75:
    train_dinner_copy['디저트2'][idx]=1
  else:
    train_dinner_copy['디저트2'][idx]=0

for idx, data in enumerate(train_dinner_copy['김치']):
  if data in kimchi_75:
    train_dinner_copy['김치'][idx]=1
  else:
    train_dinner_copy['김치'][idx]=0


/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/3505677609.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dinner_copy['밥'][idx]=1
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/3505677609.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dinner_copy['밥'][idx]=0
/var/folders/cs/4cpv5sr95s55j7dqdctl5qpm0000gn/T/ipykernel_25033/3505677609.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [80]:
train_dinner_copy.head()

,index,일자,석식메뉴,석식계,석식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,디저트1,디저트2,김치
0,0,2016-02-01,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 포기김치 (김치:국내산)",331.0,"[쌀밥/잡곡밥, 육개장, 자반고등어구이, 두부조림, 건파래무침, 포기김치]",1,1,0,0,0,1,1,1,1
1,1,2016-02-02,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무침 바나나 포기김치 (배추,고추가루:국내산)",560.0,"[콩나물밥*양념장, 어묵국, 유산슬, 아삭고추무침, 바나나, 포기김치]",1,1,1,1,1,1,1,1,1
2,2,2016-02-03,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기전 (돼지고기:국내산) 새송이버섯볶음 포기김치 (배추,고추가루:국내산)",573.0,"[쌀밥/잡곡밥, 청국장찌개, 황태양념구이, 고기전, 새송이버섯볶음, 포기김치]",1,1,1,1,1,1,1,1,1
3,3,2016-02-04,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포기김치 (배추,고추가루:국내산)",525.0,"[미니김밥*겨자장, 우동, 멕시칸샐러드, 군고구마, 무피클, 포기김치]",0,1,1,0,1,1,1,1,1
4,4,2016-02-05,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:국내산) 감자소세지볶음 콩나물무침 포기김치 (배추,고추가루:국내산)",330.0,"[쌀밥/잡곡밥, 차돌박이찌개, 닭갈비, 감자소세지볶음, 콩나물무침, 포기김치]",1,1,1,0,1,1,1,1,1


In [81]:
#test_dinner_copy
test_dinner_copy = test_dinner.copy()
for idx, data in enumerate(test_dinner_copy['밥']):
  if data in rice_75:
    test_dinner_copy['밥'][idx]=1
  else:
    test_dinner_copy['밥'][idx]=0

for idx, data in enumerate(test_dinner_copy['국']):
  if data in soup_75:
    test_dinner_copy['국'][idx]=1
  else:
    test_dinner_copy['국'][idx]=0

for idx, data in enumerate(test_dinner_copy['반찬1']):
  if data in side_1_75:
    test_dinner_copy['반찬1'][idx]=1
  else:
    test_dinner_copy['반찬1'][idx]=0

for idx, data in enumerate(test_dinner_copy['반찬2']):
  if data in side_2_75:
    test_dinner_copy['반찬2'][idx]=1
  else:
    test_dinner_copy['반찬2'][idx]=0

for idx, data in enumerate(test_dinner_copy['반찬3']):
  if data in side_3_75:
    test_dinner_copy['반찬3'][idx]=1
  else:
    test_dinner_copy['반찬3'][idx]=0    

for idx, data in enumerate(test_dinner_copy['반찬4']):
  if data in side_4_75:
    test_dinner_copy['반찬4'][idx]=1
  else:
    test_dinner_copy['반찬4'][idx]=0

for idx, data in enumerate(test_dinner_copy['디저트1']):
  if data in dessert_1_75:
    test_dinner_copy['디저트1'][idx]=1
  else:
    test_dinner_copy['디저트1'][idx]=0

for idx, data in enumerate(test_dinner_copy['디저트2']):
  if data in dessert_2_75:
    test_dinner_copy['디저트2'][idx]=1
  else:
    test_dinner_copy['디저트2'][idx]=0

for idx, data in enumerate(test_dinner_copy['김치']):
  if data in kimchi_75:
    test_dinner_copy['김치'][idx]=1
  else:
    test_dinner_copy['김치'][idx]=0


In [82]:
test_dinner_copy.head()

,일자,석식메뉴,석식메뉴리스트,밥,국,반찬1,반찬2,반찬3,반찬4,디저트1,디저트2,김치
0,2021-01-27,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,"[흑미밥, 얼큰순두부찌개, 쇠고기우엉볶음, 버섯햄볶음, 포기김치]",1,0,0,0,1,1,1,1,1
1,2021-01-28,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,"[충무김밥, 우동국물, 오징어무침, 꽃맛살샐러드, 얼갈이쌈장무침, 석박지]",1,1,1,1,0,1,1,1,1
2,2021-01-29,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치,"[흑미밥, 물만둣국, 카레찜닭, 숯불양념꼬지어묵, 꼬시래기무침, 포기김치]",1,0,0,0,1,1,1,1,1
3,2021-02-01,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치,"[흑미밥, 동태탕, 돈육꽈리고추장조림, 당면채소무침, 모자반무침, 포기김치]",1,1,0,0,0,1,1,1,1
4,2021-02-02,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치,"[흑미밥, 바지락살국, 쇠고기청경채볶음, 두부구이*볶은김치, 머위된장무침, 백김치]",1,0,1,0,0,1,1,1,1


## 메뉴만으로 학습 및 예측

In [83]:
data = train_lunch_copy.iloc[:,4:]
target = train_lunch_copy.iloc[:,2]

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

svm_poly_reg = SVR()

params = {
    'kernel':['poly'],
    'degree':[1,2,3],
    'C':[1,10,100],
    'epsilon':[0.1,1]
}

grid_svm = GridSearchCV(svm_poly_reg, param_grid=params, cv=5,
                        scoring='neg_mean_absolute_error',
                        refit=True)
grid_svm.fit(data, target)
cvres = grid_svm.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("MAE(cv=5) :",-mean_score, params)

MAE(cv=5) : 151.7909380794638 {'C': 1, 'degree': 1, 'epsilon': 0.1, 'kernel': 'poly'}
MAE(cv=5) : 151.80163855139773 {'C': 1, 'degree': 1, 'epsilon': 1, 'kernel': 'poly'}
MAE(cv=5) : 139.3987673479376 {'C': 1, 'degree': 2, 'epsilon': 0.1, 'kernel': 'poly'}
MAE(cv=5) : 139.44449754189168 {'C': 1, 'degree': 2, 'epsilon': 1, 'kernel': 'poly'}
MAE(cv=5) : 134.04207992147477 {'C': 1, 'degree': 3, 'epsilon': 0.1, 'kernel': 'poly'}
MAE(cv=5) : 134.0514734368695 {'C': 1, 'degree': 3, 'epsilon': 1, 'kernel': 'poly'}
MAE(cv=5) : 145.7978218367482 {'C': 10, 'degree': 1, 'epsilon': 0.1, 'kernel': 'poly'}
MAE(cv=5) : 145.80913417572668 {'C': 10, 'degree': 1, 'epsilon': 1, 'kernel': 'poly'}
MAE(cv=5) : 136.6401765599759 {'C': 10, 'degree': 2, 'epsilon': 0.1, 'kernel': 'poly'}
MAE(cv=5) : 136.68061645304007 {'C': 10, 'degree': 2, 'epsilon': 1, 'kernel': 'poly'}
MAE(cv=5) : 135.85555127841877 {'C': 10, 'degree': 3, 'epsilon': 0.1, 'kernel': 'poly'}
MAE(cv=5) : 135.83376740219697 {'C': 10, 'degree': 3,